# Notes:

Metrics to add:
- column for type of workout so we can predict in different workouts


Time sequence options:
- pad at the end of each sequence and build an LSTM RNN with a layer that ignores padded portions of the sequence and a bidirectional layer to retain context from each phase
    1) separate pitch into phases (wind up, sit down power, and throw maybe? or possibly: wind-up, stride, arm cocking, acceleration, release)
        - ENSURE phases are always started the same, this takes away the noise of different starts and is ESSENTIAL or just do tree based instead of lstm
        - Add phase ID (e.g., wind_up=0, acceleration=1, release=2) as a feature to the model.
    2) use time column to split into different sequences (per trial)
        - consider normalizing the time sequences to 0,1. This only makes sense if it's not already normalized, check first and check distance in time steps to ensure no noise
    3) pad at the end of each phase so that it each sequence matches the max sequence
        - Pad each phase separately to its own max length (e.g., wind-up padded to 20 steps, acceleration to 50 steps).
        - Use a hierarchical model (e.g., separate LSTM per phase) to avoid mixing padded values across phases.
    4) create masked neural network that is bidirectional for edge cases and masking so we exclude the padded steps
        For causal tasks, use unidirectional LSTMs.
        For non-causal tasks (e.g., post-pitch classification), use bidirectional layers.
            from tensorflow.keras.layers import Input, Masking, Bidirectional, LSTM, Dense

            # Phase-specific input (e.g., wind-up)
            inputs = Input(shape=(max_phase_length, num_features + 2))  # +2 for phase ID and normalized time
            masked = Masking(mask_value=-1.0)(inputs)  # Assume padded time = -1
            lstm_out = Bidirectional(LSTM(64))(masked)  
            outputs = Dense(1, activation='sigmoid')(lstm_out)
        Add attention mechanisms to explicitly highlight critical moments (e.g., release):
            python
            from tensorflow.keras.layers import Attention, Concatenate
            # Add attention to focus on key timesteps (e.g., release)
            context_vector = Attention()([lstm_out, lstm_out])
            outputs = Concatenate()([lstm_out, context_vector])


Other options to test
- Dynamic Time Warping (DTW): Align sequences non-linearly to a reference length.
    Use Case: Best for comparing shapes of biomechanical curves (e.g., elbow angle vs. time).
    Use DTW as a preprocessing step for similarity-based tasks (e.g., clustering pitches by motion pattern).

- Resampling: Interpolate sequences to a fixed length (e.g., 100 steps) using biomechanical curves’ shape.
    Use Case: Suitable when biomechanical patterns are time-invariant (e.g., joint angles during release).
    Use cubic spline interpolation instead of linear to preserve curve shape.


Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.
Key Tests to Validate Approach

    Phase Alignment Check:

        Plot the distribution of phase durations (e.g., wind-up: 0.2–0.3s, release: 0.1–0.15s).

        Ensure padding doesn’t exceed 20% of the phase’s natural duration.

    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)
    Ablation Study:

        Compare performance of:
        a) Phase-padded LSTM
        b) Resampled LSTM
        c) DTW-aligned model

    Attention Visualization:

        Use libraries like tf-explain to confirm attention focuses on biomechanically critical moments (e.g., release).

    Noise Injection Test:

        Add Gaussian noise to padded regions and verify model robustness (accuracy shouldn’t degrade)

# Load in the datasets: summarized, emg left joined, bio interpolated

In [1]:
import pandas as pd

# Load EMG pitch data for validation not for use really
emg_pitch_data = pd.read_parquet('../data/processed/emg_pitch_data_processed.parquet')
print("\nEMG Pitch Data Columns and Unique Values:")
for col in emg_pitch_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {emg_pitch_data[col].unique()}")

# Load summarized biomechanical profile
# summarized_bio = pd.read_parquet('../data/processed/ml_datasets/summarized/final_summarized_biomechanical_profile.parquet')
# print("\nSummarized Biomechanical Profile Columns and Unique Values:") 
# for col in summarized_bio.columns:
#     print(f"\n{col}:") 
#     print(f"Unique values: {summarized_bio[col].unique()}")


# Granular Datasets:
# Resample/interpolate the biomechanics dataset to join evenly onto the EMG data: No changes
# Bio dataset with EMG filtered out dataset: is_interpolated filter: filter for non interpolated data if you want to take away emg data for a bio dataset without interpolated added columns (will filter out EMG so they are on the bio frequency)
# EMG dataset with phases added on: create a interpolated column list so we can differ that from the non and create a EMG dataset with pitch phases added on (creating the simplistic EMG dataset with phases added on, no fake data involved and straight to the muscles dataset)

# Load granular dataset with interpolated bio data
granular_data = pd.read_parquet('../data/processed/ml_datasets/final_inner_join_emg_biomech_data.parquet')
print("\nGranular Dataset Columns and Unique Values:")
for col in granular_data.columns:
    print(f"\n{col}:")
    print(f"Unique values: {granular_data[col].unique()}")








EMG Pitch Data Columns and Unique Values:

EMG 1 (mV) - FDS (81770):
Unique values: [-0.0391089 -0.0345769 -0.0142672 ...  0.0040284  0.1359579  0.0691539]

ACC X (G) - FDS (81770):
Unique values: [-0.9207153 -0.9168091 -0.8994141 ...  0.4370117 -1.8967285 -1.4609985]

ACC Y (G) - FDS (81770):
Unique values: [0.406311  0.4068604 0.4025269 ... 0.1681519 0.414917  0.5172119]

ACC Z (G) - FDS (81770):
Unique values: [-0.3459473 -0.3414917 -0.3353271 ...  0.190918   0.2297974  0.1813354]

GYRO X (deg/s) - FDS (81770):
Unique values: [ 33.5343513  32.5190849  32.        ... -92.5190811  57.7557259
  44.0534363]

GYRO Y (deg/s) - FDS (81770):
Unique values: [  -6.2442746   -7.8091602   -9.2748089 ... -236.870224  -212.427475
 -107.2900772]

GYRO Z (deg/s) - FDS (81770):
Unique values: [-17.1832066 -17.4503822 -17.5496178 ...  69.3587799  64.8473282
  95.5267181]

EMG 1 (mV) - FCU (81728):
Unique values: [-0.008896   0.0159457  0.0287022 ...  0.336202   0.0241703 -0.207126 ]

ACC X (G) - FCU

EDA
Check all the basics:
- p value for each metric for suspicious stuff
- outliers
- standardization
- normalization
- dispersion
- disparity
- nulls and which to take out or to impute
- anything else that fixes this dataset 

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.decomposition import PCA

def detailed_eda(df, dataset_name="Dataset", display_plots=False):
    """
    Perform a detailed Exploratory Data Analysis (EDA) on the given DataFrame.
    
    Additional features added:
    - Duplicate row checks.
    - Consistency checks for datetime columns.
    - Visual Exploratory Analysis (histograms, density plots, Q-Q plots, boxplots).
    - Multicollinearity analysis using Variance Inflation Factor (VIF).
    - Basic time series checks: Stationarity (ADF test) and ACF/PACF plots.
    - PCA analysis for dimensionality reduction.
    - Domain-specific range checks for EMG, acceleration, and rotational columns.
    
    Parameters:
    - df: pandas DataFrame to analyze.
    - dataset_name: a string name for the dataset (used in print statements).
    - display_plots: if True, displays matplotlib plots (set to False for non-interactive environments).
    """
    print(f"\n\n----- Detailed EDA for {dataset_name} -----")
    
    # ---------------------------
    # Basic Data Overview
    # ---------------------------
    print("Shape:", df.shape)
    
    # Duplicate row check
    dup_count = df.duplicated().sum()
    print(f"\nDuplicate Rows: {dup_count} duplicate rows found.")
    
    print("\nDataFrame Info:")
    df.info()
    
    print("\nDescriptive Statistics:")
    print(df.describe(include='all').T)
    
    # ---------------------------
    # Nulls Check
    # ---------------------------
    print("\nMissing Values:")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if missing.empty:
        print("No missing values found.")
    else:
        print(missing)
    
    # ---------------------------
    # Identify Numeric and Datetime Columns
    # ---------------------------
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    # Filter out timedelta columns if any
    numeric_cols = [col for col in numeric_cols if not pd.api.types.is_timedelta64_dtype(df[col])]
    
    datetime_cols = df.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()
    
    # ---------------------------
    # Check for Datetime Consistency
    # ---------------------------
    if datetime_cols:
        print("\nDatetime Consistency Checks:")
        for col in datetime_cols:
            # Check if the datetime column is sorted
            if not df[col].is_monotonic_increasing:
                print(f"Warning: The datetime column '{col}' is not in increasing order.")
            else:
                print(f"The datetime column '{col}' is properly ordered.")
    
    # ---------------------------
    # Normality Test (Shapiro-Wilk) for Numeric Columns
    # ---------------------------
    print("\nNormality Test (Shapiro-Wilk) for numeric columns (p-values):")
    for col in numeric_cols:
        try:
            data = df[col].dropna()
            if len(data) > 5000:
                data = data.sample(5000, random_state=42)
            stat, p_value = stats.shapiro(data)
            print(f"{col}: p-value = {p_value:.4f}")
        except Exception as e:
            print(f"{col}: Error in normality test: {e}")
    
    # ---------------------------
    # Outlier Detection using IQR Method
    # ---------------------------
    print("\nOutlier Detection using IQR method:")
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        print(f"{col}: {outlier_count} outliers detected.")
    
    # ---------------------------
    # Demonstrate Standardization and Normalization
    # ---------------------------
    print("\nStandardization (first 5 rows, Z-scores):")
    standardized = df[numeric_cols].apply(lambda x: (x - x.mean()) / x.std())
    print(standardized.head())
    
    print("\nNormalization (first 5 rows, scaled between 0 and 1):")
    normalized = df[numeric_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    print(normalized.head())
    
    # ---------------------------
    # Dispersion Metrics for Numeric Columns
    # ---------------------------
    print("\nDispersion and Disparity Metrics for numeric columns:")
    for col in numeric_cols:
        mean_val = df[col].mean()
        std_val = df[col].std()
        var_val = df[col].var()
        min_val = df[col].min()
        max_val = df[col].max()
        range_val = max_val - min_val
        print(f"{col}: mean = {mean_val:.4f}, std = {std_val:.4f}, variance = {var_val:.4f}, range = {range_val:.4f}")
    
    # ---------------------------
    # Correlation Matrix
    # ---------------------------
    print("\nCorrelation Matrix (numeric columns):")
    corr_matrix = df[numeric_cols].corr()
    print(corr_matrix)
    
    # ---------------------------
    # Visual Exploratory Analysis
    # ---------------------------
    print("\nGenerating visual exploratory plots (if display_plots=True)...")
    if display_plots:
        for col in numeric_cols:
            plt.figure(figsize=(12, 4))
            
            # Histogram and density plot
            plt.subplot(1, 3, 1)
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(f'Histogram & KDE: {col}')
            
            # Q-Q plot
            plt.subplot(1, 3, 2)
            stats.probplot(df[col].dropna(), dist="norm", plot=plt)
            plt.title(f'Q-Q Plot: {col}')
            
            # Boxplot
            plt.subplot(1, 3, 3)
            sns.boxplot(x=df[col].dropna())
            plt.title(f'Boxplot: {col}')
            
            plt.tight_layout()
            plt.show()
    
    # ---------------------------
    # Multicollinearity Check using VIF
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nMulticollinearity Check using VIF:")
        # Prepare data for VIF calculation (drop missing values)
        df_numeric = df[numeric_cols].dropna()
        vif_data = pd.DataFrame()
        vif_data["Feature"] = df_numeric.columns
        vif_data["VIF"] = [variance_inflation_factor(df_numeric.values, i) 
                           for i in range(len(df_numeric.columns))]
        print(vif_data)
    
    # ---------------------------
    # Time Series Specific Checks (if applicable)
    # ---------------------------
    if datetime_cols:
        print("\nTime Series Analysis:")
        # For each datetime column, try to find an associated numeric column for time series checks.
        for dt_col in datetime_cols:
            # Find numeric columns that might be associated (heuristic: columns that are not IDs)
            candidate_cols = [col for col in numeric_cols if 'id' not in col.lower()]
            for num_col in candidate_cols:
                # Sort by datetime column
                ts_data = df[[dt_col, num_col]].dropna().sort_values(by=dt_col)
                if ts_data.shape[0] > 30:  # ensure enough data points
                    # Stationarity test: ADF test
                    try:
                        adf_result = adfuller(ts_data[num_col])
                        print(f"ADF Test for {num_col} (sorted by {dt_col}): p-value = {adf_result[1]:.4f}")
                    except Exception as e:
                        print(f"Error performing ADF test on {num_col}: {e}")
                    
                    # If display_plots is True, plot ACF and PACF
                    if display_plots:
                        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
                        plot_acf(ts_data[num_col], ax=axes[0], lags=20)
                        axes[0].set_title(f'ACF of {num_col}')
                        plot_pacf(ts_data[num_col], ax=axes[1], lags=20)
                        axes[1].set_title(f'PACF of {num_col}')
                        plt.tight_layout()
                        plt.show()
                    # Only process one candidate per datetime column
                    break

    # ---------------------------
    # PCA Analysis for Numeric Data
    # ---------------------------
    if len(numeric_cols) > 1:
        print("\nPCA Analysis on numeric columns:")
        # Drop NA values for PCA and standardize data
        pca_data = df[numeric_cols].dropna()
        standardized_data = (pca_data - pca_data.mean()) / pca_data.std()
        try:
            pca = PCA()
            pca.fit(standardized_data)
            explained_variance = pca.explained_variance_ratio_
            for idx, var in enumerate(explained_variance):
                print(f"PC{idx+1}: {var*100:.2f}% of variance explained")
        except Exception as e:
            print("Error performing PCA:", e)
    
    # ---------------------------
    # Domain-Specific Range Checks
    # ---------------------------
    print("\nDomain-Specific Range Checks:")
    domain_keywords = ['emg', 'accel', 'rotat']
    for col in numeric_cols:
        if any(keyword in col.lower() for keyword in domain_keywords):
            col_min = df[col].min()
            col_max = df[col].max()
            print(f"{col} (domain-specific): min = {col_min}, max = {col_max}")
            # Here you can add further domain-specific validations if needed.
    
    print("\n----- End of EDA for", dataset_name, "-----\n")


# ------------------ New EDA Calls ------------------
# Run the detailed EDA on each dataset
# detailed_eda(emg_pitch_data, dataset_name="EMG Pitch Data")
# detailed_eda(summarized_bio, dataset_name="Summarized Biomechanical Profile")
detailed_eda(granular_data, dataset_name="Granular Dataset")




----- Detailed EDA for Granular Dataset -----
Shape: (134720, 115)

Duplicate Rows: 0 duplicate rows found.

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134720 entries, 0 to 134719
Columns: 115 entries, EMG 1 (mV) - FDS (81770) to time_difference
dtypes: bool(1), datetime64[ns](4), float64(69), int64(19), object(18), timedelta64[ns](4)
memory usage: 117.3+ MB

Descriptive Statistics:
                                 count unique         top    freq  \
EMG 1 (mV) - FDS (81770)      134720.0    NaN         NaN     NaN   
ACC X (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
ACC Y (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
ACC Z (G) - FDS (81770)       134720.0    NaN         NaN     NaN   
GYRO X (deg/s) - FDS (81770)  134720.0    NaN         NaN     NaN   
...                                ...    ...         ...     ...   
time_step_biomech               134720    NaN         NaN     NaN   
date_biomech                    13472

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


ACC X (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC X (G) - FCU (81728)_spike_flag: p-value = 0.0000
ACC Y (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC Y (G) - FCU (81728)_spike_flag: p-value = 0.0000
ACC Z (G) - FDS (81770)_spike_flag: p-value = 0.0000
ACC Z (G) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO X (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO X (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO Y (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO Y (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
GYRO Z (deg/s) - FDS (81770)_spike_flag: p-value = 0.0000
GYRO Z (deg/s) - FCU (81728)_spike_flag: p-value = 0.0000
EMG 1 (mV) - FDS (81770)_spike_flag: p-value = 0.0000
EMG_high_flag: p-value = 0.0000
EMG_low_flag: p-value = 0.0000
EMG_extreme_flag: p-value = 0.0000
EMG_extreme_flag_dynamic: p-value = 1.0000
ThrowingMotion: p-value = 0.0000
session_biomech: p-value = 1.0000
trial_biomech: p-value = 0.0000
ongoing_timestamp_biomech: p-value = 1.0

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                                        Feature       VIF
0                      EMG 1 (mV) - FDS (81770)  1.000002
1                       ACC X (G) - FDS (81770)  1.000055
2                       ACC Y (G) - FDS (81770)  1.000010
3                       ACC Z (G) - FDS (81770)  1.000000
4                  GYRO X (deg/s) - FDS (81770)  1.000110
..                                          ...       ...
83            phase_weighted_cumulative_biomech  1.002456
84                    cumulative_valgus_biomech  1.000630
85                       critical_phase_biomech  1.022390
86  cumulative_valgus_phase_armcock_acc_biomech  1.101465
87                   peak_torque_marker_biomech  1.117854

[88 rows x 2 columns]

Time Series Analysis:
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Date/Time_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by Timestamp_parsed): p-value = 0.0000
ADF Test for EMG 1 (mV) - FDS (81770) (sorted by datetime): p-value = 0.0000
ADF Test fo

# Feature Engineering

metrics to add:
* Set up phases based on the pitcher movement so we can set them into different phases
* add the phases to the EMG dataset so we can better predict on that and create use the merge_asof dataset to see if it's worth predicting with for the time series motion of the joints (the merge_asof would need to be a small difference)
* add in pulse data if we don't have arm slot/torque/speed in the database
* add in vulgas range and possibly the ongoing detriment from that
* finish up the sql statement (ensure it's completely validated) for the base table
* separate the dataset into time series classification, regression and tree based classification datasets
* check the feaures importances and such
* use permutation importance and use github(dot)com/eli5-org/eli5

Setting up feature dictionary to speak over each feature to talk about actual important metrics outside of those feature selected
Feature Selection (RFE, SHAP importance, correlation (to take out multi-collinearity metrics or combine them))



y metric ideas:
Can we also add in muscle metrics like a muscle contraction score between all muscles available and an exhaustion rate based on that. Can we also add in extreme range of motion metrics for each of the the joints. Ensure to go through the metrics and see if we can add anything else for features. 



In [6]:
from itertools import combinations
import numpy as np
import pandas as pd
from scipy import stats, signal
import pywt
from scipy.stats import entropy

# -----------------------------
# ACWR Function (Existing)
# -----------------------------
def compute_acwr(workload_series, acute_window=7, chronic_window=28):
    """
    Compute the acute-to-chronic workload ratio (ACWR) for a given workload time series.
    
    Parameters:
      workload_series (pd.Series): A time series of workload values.
      acute_window (int): The number of time units (e.g. days) to use for calculating the acute load.
      chronic_window (int): The number of time units (e.g. days) to use for calculating the chronic load.
    
    Returns:
      pd.Series: The ACWR values computed as (acute load)/(chronic load).
    """
    acute_load = workload_series.rolling(window=acute_window, min_periods=1).sum()
    chronic_load = workload_series.rolling(window=chronic_window, min_periods=1).mean()
    chronic_load = chronic_load.replace(0, np.nan)
    acwr = acute_load / chronic_load
    return acwr

# -----------------------------
# New Helper Function for ACWR on DataFrames
# -----------------------------
def compute_acwr_df(df, acute_window=7, chronic_window=28):
    """
    Apply the compute_acwr function to each column in a DataFrame.
    
    Parameters:
      df (pd.DataFrame): A DataFrame where each column is a workload time series (e.g. per joint or per muscle).
      acute_window (int): The acute window for computing ACWR.
      chronic_window (int): The chronic window for computing ACWR.
      
    Returns:
      pd.DataFrame: A DataFrame of ACWR values with the same index as df and columns named "ACWR_<original column>".
    """
    acwr_dict = {}
    for col in df.columns:
        acwr_dict["ACWR_" + col] = compute_acwr(df[col], acute_window=acute_window, chronic_window=chronic_window)
    return pd.DataFrame(acwr_dict, index=df.index)

# -----------------------------
# New Functions
# -----------------------------
def muscle_contraction_score(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Calculate a muscle contraction coordination score between all available muscles.
    Higher scores indicate better coordination, while lower scores suggest fatigue-related disruption.
    
    Steps:
      - Normalize each muscle's EMG by its maximum value.
      - In each sliding window, compute pairwise correlation coefficients.
      - Average these correlations to form a coordination score.
      - Also compute the standard deviation (variability) for each muscle's activation.
    
    Returns:
      DataFrame with:
        - 'muscle_coordination_score'
        - Individual contraction variability for each muscle
        - A composite contraction variability metric.
    """
    results = pd.DataFrame()
    normalized_emg = pd.DataFrame()
    for muscle in muscle_cols:
        max_activation = emg_data[muscle].max()
        normalized_emg[muscle] = emg_data[muscle] / (max_activation if max_activation != 0 else 1)
    muscle_pairs = list(combinations(muscle_cols, 2))
    for i in range(window_size, len(emg_data)+1, window_size // 2):
        segment = normalized_emg.iloc[max(0, i-window_size):i]
        pair_correlations = []
        for muscle1, muscle2 in muscle_pairs:
            corr = segment[muscle1].corr(segment[muscle2])
            pair_correlations.append(corr)
        avg_correlation = np.nanmean(pair_correlations)
        results.loc[i, 'muscle_coordination_score'] = avg_correlation
        variability_list = []
        for muscle in muscle_cols:
            std_val = segment[muscle].std()
            results.loc[i, f'{muscle}_contraction_variability'] = std_val
            variability_list.append(std_val)
        results.loc[i, 'composite_contraction_variability'] = np.mean(variability_list)
    return results

def exhaustion_rate(contraction_score_data, decay_data, freq_shift_data, window_size=60):
    """
    Calculate the exhaustion rate based on multiple fatigue indicators.
    
    For each window:
      - Compute the rate of change (slope) of the muscle coordination score.
      - Average the decay rate and compute a frequency shift factor.
      - Combine these metrics using weighted coefficients to produce an exhaustion rate.
      
    Returns:
      DataFrame with a column 'exhaustion_rate' computed for each window.
    """
    results = pd.DataFrame()
    for i in range(window_size, len(contraction_score_data)+1, window_size // 2):
        segment_idx = range(max(0, i-window_size), i)
        y_vals = contraction_score_data.loc[segment_idx, 'muscle_coordination_score']
        if y_vals.dropna().empty or len(y_vals.dropna()) < 2:
            coord_slope = np.nan
        else:
            coord_slope = stats.linregress(np.arange(len(y_vals.dropna())), y_vals.dropna())[0]
        avg_decay = decay_data.loc[segment_idx, 'composite_decay_rate'].mean()
        avg_freq_shift = 1 - freq_shift_data.loc[segment_idx, 'composite_freq_shift'].mean()
        exhaustion = (-coord_slope * 0.4) + (abs(avg_decay) * 0.3) + (avg_freq_shift * 0.3)
        results.loc[i, 'exhaustion_rate'] = exhaustion
    return results

def extract_wavelet_features(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Extract wavelet-based features from EMG signals.
    
    For each muscle and for each sliding window:
      - Perform a wavelet decomposition using the 'db4' wavelet.
      - For each decomposition level, compute:
          - The energy (sum of squared coefficients)
          - The entropy (based on the absolute coefficient values)
          
    Returns:
      DataFrame with features named as:
        '{muscle}_wavelet_energy_level_{level}'
        '{muscle}_wavelet_entropy_level_{level}'
    """
    results = pd.DataFrame()
    for muscle in muscle_cols:
        for i in range(window_size, len(emg_data)+1, window_size // 2):
            segment = emg_data[muscle].iloc[max(0, i-window_size):i].values
            if len(segment) < window_size // 2:
                continue
            coeffs = pywt.wavedec(segment, 'db4', level=4)
            for level, coef in enumerate(coeffs):
                energy = np.sum(coef**2)
                coef_abs = np.abs(coef)
                prob_dist = coef_abs / (np.sum(coef_abs) + 1e-6)
                coef_entropy = entropy(prob_dist + 1e-6)
                results.loc[i, f'{muscle}_wavelet_energy_level_{level}'] = energy
                results.loc[i, f'{muscle}_wavelet_entropy_level_{level}'] = coef_entropy
    return results

def create_progression_features(fatigue_data, window_sizes=[5, 10, 20, 30]):
    """
    Create features that capture the progression of fatigue metrics over multiple time scales.
    
    For each numeric column in the fatigue_data:
      - Compute a rolling mean (trend) for each specified window.
      - Compute a rolling standard deviation (variability) for each window.
      - Compute the rate of change (first derivative) and acceleration (second derivative).
      
    Returns:
      DataFrame containing the progression features.
    """
    results = pd.DataFrame(index=fatigue_data.index)
    for col in fatigue_data.columns:
        if not np.issubdtype(fatigue_data[col].dtype, np.number):
            continue
        for window in window_sizes:
            results[f'{col}_trend_{window}'] = fatigue_data[col].rolling(window=window, min_periods=1).mean()
            results[f'{col}_variability_{window}'] = fatigue_data[col].rolling(window=window, min_periods=1).std()
            results[f'{col}_rate_{window}'] = fatigue_data[col].diff(periods=window) / window
            results[f'{col}_acceleration_{window}'] = results[f'{col}_rate_{window}'].diff(periods=window) / window
    return results

# -----------------------------
# Existing Functions (Activation, Frequency, etc.)
# -----------------------------
def activation_decay_index(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Calculate the normalized rate of EMG amplitude decline for each muscle.
    """
    results = pd.DataFrame()
    for muscle in muscle_cols:
        print(f"[DEBUG] Muscle {muscle} | Raw data summary:")
        print(emg_data[muscle].describe())
        initial_activation = emg_data[muscle].iloc[:window_size].mean()
        print(f"[DEBUG] Muscle {muscle} | Initial activation (first {window_size} values): {initial_activation:.4f}")
        rolling_activation = emg_data[muscle].rolling(window=window_size, min_periods=window_size//2).mean()
        print(f"[DEBUG] Muscle {muscle} | Rolling activation (first 10 values):")
        print(rolling_activation.head(10))
        normalized_activation = rolling_activation / initial_activation
        print(f"[DEBUG] Muscle {muscle} | Normalized activation stats:")
        print(normalized_activation.describe())
        decay_rates = []
        for i in range(window_size, len(emg_data), window_size // 2):
            segment = normalized_activation.iloc[max(0, i-window_size):i]
            print(f"[DEBUG] Muscle {muscle} | Segment {i-window_size} to {i} values:")
            print(segment.values)
            valid_count = segment.dropna().shape[0]
            print(f"[DEBUG] Muscle {muscle} | Segment {i-window_size} to {i}: valid_count = {valid_count}")
            print(f"[DEBUG] Muscle {muscle} | Segment {i-window_size} to {i} summary:")
            print(segment.describe())
            if valid_count >= window_size // 2:
                valid_mask = segment.notna()
                x_vals = np.arange(len(segment))[valid_mask]
                seg_vals = segment.dropna().values
                print(f"[DEBUG] Muscle {muscle} | x_vals: {x_vals}")
                print(f"[DEBUG] Muscle {muscle} | seg_vals: {seg_vals}")
                slope, _, _, _, _ = stats.linregress(x_vals, seg_vals)
                print(f"[DEBUG] Muscle {muscle} | Computed slope = {slope}")
                decay_rates.append(slope)
            else:
                print(f"[DEBUG] Muscle {muscle} | Insufficient valid data, appending NaN")
                decay_rates.append(np.nan)
        results[f'{muscle}_decay_rate'] = decay_rates
    results['composite_decay_rate'] = results.mean(axis=1)
    return results

def frequency_shift_index(emg_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60, sampling_rate=1000):
    """
    Calculate the shift in median frequency of EMG signals as an indicator of fatigue.
    """
    results = pd.DataFrame()
    for muscle in muscle_cols:
        median_freqs = []
        for i in range(window_size, len(emg_data), window_size // 2):
            segment = emg_data[muscle].iloc[max(0, i-window_size):i].values
            if len(segment) >= window_size // 2:
                f, Pxx = signal.welch(segment, fs=sampling_rate, nperseg=min(256, len(segment)))
                total_power = np.sum(Pxx)
                cumulative_power = np.cumsum(Pxx)
                median_freq_idx = np.where(cumulative_power >= total_power / 2)[0][0]
                median_freq = f[median_freq_idx]
                median_freqs.append(median_freq)
            else:
                median_freqs.append(np.nan)
        results[f'{muscle}_median_freq'] = median_freqs
    for muscle in muscle_cols:
        initial_freq = results[f'{muscle}_median_freq'].iloc[0]
        results[f'{muscle}_freq_ratio'] = results[f'{muscle}_median_freq'] / initial_freq
    results['composite_freq_shift'] = results[[f'{muscle}_freq_ratio' for muscle in muscle_cols]].mean(axis=1)
    return results

def force_activation_efficiency(emg_data, force_data, muscle_cols=['FDS', 'FCU', 'FCR'], window_size=60):
    """
    Calculate the efficiency ratio between force output and EMG activation.
    """
    results = pd.DataFrame()
    for muscle in muscle_cols:
        efficiency_values = []
        for i in range(window_size, len(emg_data), window_size // 2):
            emg_segment = emg_data[muscle].iloc[max(0, i-window_size):i]
            force_segment = force_data.iloc[max(0, i-window_size):i]
            if len(emg_segment) >= window_size // 2:
                mean_activation = emg_segment.mean()
                mean_force = force_segment.mean()
                efficiency = mean_force / (mean_activation + 1e-6)
                efficiency_values.append(efficiency)
            else:
                efficiency_values.append(np.nan)
        results[f'{muscle}_efficiency'] = efficiency_values
    for muscle in muscle_cols:
        initial_efficiency = results[f'{muscle}_efficiency'].iloc[0]
        results[f'{muscle}_efficiency_ratio'] = results[f'{muscle}_efficiency'] / initial_efficiency
    results['composite_efficiency'] = results[[f'{muscle}_efficiency_ratio' for muscle in muscle_cols]].mean(axis=1)
    return results

def composite_exhaustion_index(decay_data, freq_data, efficiency_data, weights=[0.4, 0.3, 0.3]):
    """
    Create a comprehensive exhaustion index by combining multiple fatigue indicators.
    """
    inverted_freq_shift = 1 - freq_data['composite_freq_shift']
    inverted_efficiency = 1 - efficiency_data['composite_efficiency']
    norm_decay = np.abs(decay_data['composite_decay_rate']) / np.abs(decay_data['composite_decay_rate']).min()
    exhaustion_index = (weights[0] * norm_decay +
                        weights[1] * inverted_freq_shift +
                        weights[2] * inverted_efficiency)
    return exhaustion_index

def joint_angle_extremity_index(kinematic_data, joint_cols, percentile_threshold=90):
    """
    Calculate the percentage of time spent near extremes of range of motion (ROM) for each joint.
    """
    results = pd.DataFrame()
    for joint in joint_cols:
        max_threshold = np.percentile(kinematic_data[joint], percentile_threshold)
        min_threshold = np.percentile(kinematic_data[joint], 100 - percentile_threshold)
        extreme_max_time = (kinematic_data[joint] >= max_threshold).mean() * 100
        extreme_min_time = (kinematic_data[joint] <= min_threshold).mean() * 100
        total_extreme_time = extreme_max_time + extreme_min_time
        results[f'{joint}_extremity_index'] = [total_extreme_time]
    results['composite_extremity_index'] = results.mean(axis=1)
    return results

def rom_progression_metric(kinematic_data, joint_cols, window_size=60):
    """
    Track the progression of maximum joint angles over time.
    """
    results = pd.DataFrame()
    for joint in joint_cols:
        max_angles = []
        for i in range(window_size, len(kinematic_data), window_size // 2):
            segment = kinematic_data[joint].iloc[max(0, i-window_size):i]
            if len(segment) >= window_size // 2:
                max_angle = segment.max()
                max_angles.append(max_angle)
            else:
                max_angles.append(np.nan)
        results[f'{joint}_max_angle'] = max_angles
        if len(max_angles) > 1 and max_angles[0] != 0:
            results[f'{joint}_rom_progression'] = [angle / max_angles[0] - 1 for angle in max_angles]
        else:
            results[f'{joint}_rom_progression'] = [0] * len(max_angles)
    results['composite_rom_progression'] = results[[f'{joint}_rom_progression' for joint in joint_cols]].mean(axis=1)
    return results

def calculate_muscle_length(kinematic_data, joints, muscle):
    """
    Placeholder function to calculate muscle length based on joint angles.
    """
    return kinematic_data[joints].sum(axis=1)

def muscle_overextension_index(kinematic_data, joint_muscle_map, rest_lengths, critical_threshold=1.2):
    """
    Calculate the percentage of time muscles spend beyond a critical length threshold.
    """
    results = pd.DataFrame()
    muscle_lengths = {}
    for muscle, joints in joint_muscle_map.items():
        muscle_lengths[muscle] = calculate_muscle_length(kinematic_data, joints, muscle)
    for muscle, length_series in muscle_lengths.items():
        critical_length = rest_lengths[muscle] * critical_threshold
        overextension_time = (length_series >= critical_length).mean() * 100
        results[f'{muscle}_overextension_index'] = [overextension_time]
        max_length_ratio = length_series.max() / rest_lengths[muscle]
        results[f'{muscle}_max_length_ratio'] = [max_length_ratio]
    results['composite_overextension_index'] = results[[f'{muscle}_overextension_index' for muscle in joint_muscle_map.keys()]].mean(axis=1)
    return results

def velocity_at_extreme_rom(kinematic_data, joint_cols, velocity_cols, percentile_threshold=90):
    """
    Calculate angular velocities when joints are at extreme ranges of motion.
    """
    results = pd.DataFrame()
    for joint, velocity in zip(joint_cols, velocity_cols):
        extreme_threshold = np.percentile(kinematic_data[joint], percentile_threshold)
        extreme_rom_mask = kinematic_data[joint] >= extreme_threshold
        if extreme_rom_mask.sum() > 0:
            mean_velocity = kinematic_data.loc[extreme_rom_mask, velocity].abs().mean()
            max_velocity = kinematic_data.loc[extreme_rom_mask, velocity].abs().max()
        else:
            mean_velocity = 0
            max_velocity = 0
        results[f'{joint}_mean_velocity_at_extreme'] = [mean_velocity]
        results[f'{joint}_max_velocity_at_extreme'] = [max_velocity]
    results['composite_velocity_at_extreme'] = results[[f'{joint}_mean_velocity_at_extreme' for joint in joint_cols]].mean(axis=1)
    return results

def composite_fatigue_units(granular_data, exhaustion_index, extremity_index, 
                            force_col="Force", time_interval_col=None, default_interval=15, 
                            recovery_factor=0.1, velocity_col=None, baseline_velocity=92.16, 
                            weights=[0.4, 0.3, 0.3]):
    """
    Original function to calculate cumulative Fatigue Units (FU) using a static baseline.
    """
    forces = granular_data[force_col]
    max_force = forces.max()
    normalized_forces = forces / max_force
    if time_interval_col and time_interval_col in granular_data.columns:
        intervals = granular_data[time_interval_col]
    else:
        intervals = pd.Series(default_interval, index=granular_data.index)
    if velocity_col and velocity_col in granular_data.columns:
        velocity_factor = granular_data[velocity_col] / baseline_velocity
    else:
        velocity_factor = pd.Series(1.0, index=granular_data.index)
    if 'composite_extremity_index' in extremity_index.columns:
        composite_ext_index = extremity_index['composite_extremity_index'].iloc[0]
    else:
        composite_ext_index = 0
    if len(exhaustion_index) != len(granular_data):
        exhaustion_index_series = pd.Series(exhaustion_index.iloc[-1], index=granular_data.index)
    else:
        exhaustion_index_series = exhaustion_index
    w1, w2, w3 = weights
    per_throw_fatigue = (w1 * exhaustion_index_series +
                         w2 * (composite_ext_index / 100) +
                         w3 * normalized_forces)
    fatigue_contrib = per_throw_fatigue * (default_interval / intervals) * velocity_factor
    cumulative_fatigue = fatigue_contrib.cumsum() - recovery_factor * np.arange(len(granular_data))
    return cumulative_fatigue

def arm_slot_categorical_metric(kinematic_data, angle_col="elbow_angle_x_biomech"):
    """
    Categorize the arm slot based on the elbow angle (x-axis) using predefined thresholds.
    """
    bins = [-np.inf, 30, 60, 90, np.inf]
    labels = ["submarine", "sidearm", "three-quarter", "overhead"]
    arm_slot_categories = pd.cut(kinematic_data[angle_col], bins=bins, labels=labels)
    return arm_slot_categories

def injury_indicator_fatigue_units_percentile(fatigue_units, percentile=95):
    """
    Returns a binary flag (1 = injury risk) if fatigue units exceed the given percentile threshold.
    """
    threshold = np.percentile(fatigue_units.dropna(), percentile)
    print(f"Computed fatigue threshold at the {percentile}th percentile: {threshold:.3f}")
    return (fatigue_units > threshold).astype(int)

# -----------------------------
# New Automated Baseline Functions
# -----------------------------
def calculate_individual_baseline(pitcher_data, velocity_col="Pitch Velocity (mph)", percentile=90):
    if len(pitcher_data) >= 5:
        baseline = pitcher_data[velocity_col].quantile(percentile / 100.0)
    else:
        baseline = pitcher_data[velocity_col].mean()
    print(f"Calculated individual baseline (percentile {percentile}) = {baseline:.2f} mph")
    return baseline

def calculate_rolling_baseline(pitcher_data, velocity_col="Pitch Velocity (mph)", window_size=5, percentile=90):
    if 'date' in pitcher_data.columns:
        pitcher_data = pitcher_data.sort_values('date')
    rolling_baselines = pitcher_data[velocity_col].rolling(window=window_size).quantile(percentile / 100.0)
    current_baseline = rolling_baselines.iloc[-1]
    print(f"Current rolling baseline (window size {window_size}, {percentile}th percentile): {current_baseline:.2f} mph")
    return current_baseline

def calculate_recovery_adjusted_baseline(pitcher_data, velocity_col="Pitch Velocity (mph)", days_rest_col="Days Rest", percentile=90):
    recovery_baselines = {}
    recovery_groups = pitcher_data.groupby(days_rest_col)
    for days, group in recovery_groups:
        if len(group) >= 3:
            recovery_baselines[days] = group[velocity_col].quantile(percentile / 100.0)
    recovery_baselines['default'] = pitcher_data[velocity_col].quantile(percentile / 100.0)
    print(f"Recovery-adjusted baselines: {recovery_baselines}")
    return recovery_baselines

def get_automated_baseline(pitcher_id, pitch_data, velocity_col="Pitch Velocity (mph)", 
                           current_rest_days=None, method="individual", window_size=5):
    pitcher_data = pitch_data[pitch_data['pitcher_id'] == pitcher_id]
    if pitcher_data.empty:
        print("No data found for pitcher_id:", pitcher_id)
        return None
    if method == "individual":
        baseline_velocity = calculate_individual_baseline(pitcher_data, velocity_col=velocity_col, percentile=90)
    elif method == "rolling":
        baseline_velocity = calculate_rolling_baseline(pitcher_data, velocity_col=velocity_col, window_size=window_size, percentile=90)
    elif method == "recovery" and current_rest_days is not None:
        baselines = calculate_recovery_adjusted_baseline(pitcher_data, velocity_col=velocity_col, days_rest_col="Days Rest", percentile=90)
        baseline_velocity = baselines.get(current_rest_days, baselines.get('default'))
    else:
        baseline_velocity = pitch_data[velocity_col].quantile(0.9)
    print(f"Automated baseline for pitcher {pitcher_id} = {baseline_velocity:.2f} mph using method '{method}'")
    return baseline_velocity

# -----------------------------
# Updated Fatigue Units Function with Automated Baseline
# -----------------------------
def improved_fatigue_units(granular_data, exhaustion_index, extremity_index,
                           force_col="valgus_torque_biomech", 
                           time_interval_col="ongoing_timestamp_biomech",
                           default_interval=15, recovery_factor=0.1,
                           velocity_col="Pitch Velocity (mph)",
                           pitcher_id=None, all_pitch_data=None, 
                           weights=[0.4, 0.3, 0.3]):
    if pitcher_id is not None and all_pitch_data is not None:
        baseline_velocity = get_automated_baseline(pitcher_id, all_pitch_data, velocity_col=velocity_col, method="individual")
    else:
        baseline_velocity = 92.16
        print("Warning: Using population average baseline. Provide pitcher_id and all_pitch_data for individual baseline.")
    forces = granular_data[force_col]
    max_force = forces.max()
    normalized_forces = forces / max_force
    if time_interval_col and time_interval_col in granular_data.columns:
        intervals = granular_data[time_interval_col]
    else:
        intervals = pd.Series(default_interval, index=granular_data.index)
    if velocity_col and velocity_col in granular_data.columns:
        velocity_factor = granular_data[velocity_col] / baseline_velocity
    else:
        velocity_factor = pd.Series(1.0, index=granular_data.index)
    if 'composite_extremity_index' in extremity_index.columns:
        composite_ext_index = extremity_index['composite_extremity_index'].iloc[0]
    else:
        composite_ext_index = 0
    if len(exhaustion_index) != len(granular_data):
        exhaustion_index_series = pd.Series(exhaustion_index.iloc[-1], index=granular_data.index)
    else:
        exhaustion_index_series = exhaustion_index
    w1, w2, w3 = weights
    per_throw_fatigue = (w1 * exhaustion_index_series +
                         w2 * (composite_ext_index / 100) +
                         w3 * normalized_forces)
    fatigue_contrib = per_throw_fatigue * (default_interval / intervals) * velocity_factor
    cumulative_fatigue = fatigue_contrib.cumsum() - recovery_factor * np.arange(len(granular_data))
    return cumulative_fatigue

# -----------------------------
# Main Testing Section
# -----------------------------
if __name__ == "__main__":
    num_samples = 100
    granular_data = pd.DataFrame({
        "elbow_angle_x_biomech": np.random.uniform(20, 100, num_samples),
        "elbow_angle_y_biomech": np.random.uniform(20, 100, num_samples),
        "elbow_angle_z_biomech": np.random.uniform(20, 130, num_samples),
        "valgus_torque_biomech": np.random.uniform(10, 100, num_samples),
        "ongoing_timestamp_biomech": np.linspace(15, 15, num_samples),
        "Pitch Velocity (mph)": np.random.uniform(88, 100, num_samples)
    })
    granular_data["pitcher_id"] = np.random.choice([101, 102], size=num_samples)
    granular_data["arm_slot"] = arm_slot_categorical_metric(granular_data, angle_col="elbow_angle_x_biomech")
    print(granular_data[["elbow_angle_x_biomech", "arm_slot"]].head())
    
    emg_cols_FDS = "EMG 1 (mV) - FDS (81770)"
    emg_cols_FCU = "EMG 1 (mV) - FCU (81728)"
    emg_cols_FCR = "EMG 1 (mV) - FCR (81745)"
    granular_data[emg_cols_FDS] = np.random.uniform(0.5, 2.0, num_samples)
    granular_data[emg_cols_FCU] = np.random.uniform(0.5, 2.0, num_samples)
    granular_data[emg_cols_FCR] = np.random.uniform(0.5, 2.0, num_samples)
    muscle_cols = [emg_cols_FDS, emg_cols_FCU, emg_cols_FCR]
    
    decay_data = activation_decay_index(granular_data, muscle_cols=muscle_cols, window_size=60)
    freq_data = frequency_shift_index(granular_data, muscle_cols=muscle_cols, window_size=60, sampling_rate=1000)
    if "Force (N)" in granular_data.columns:
        force_data = granular_data["Force (N)"]
    else:
        force_data = pd.Series(np.ones(len(granular_data)), index=granular_data.index)
    efficiency_data = force_activation_efficiency(granular_data, force_data, muscle_cols=muscle_cols, window_size=60)
    
    exhaustion_index = composite_exhaustion_index(decay_data, freq_data, efficiency_data)
    contraction_score_data = muscle_contraction_score(granular_data, muscle_cols=muscle_cols, window_size=60)
    exhaustion_rate_data = exhaustion_rate(contraction_score_data, decay_data, freq_data, window_size=60)
    wavelet_features = extract_wavelet_features(granular_data, muscle_cols=muscle_cols, window_size=60)
    
    joint_cols = ["elbow_angle_x_biomech", "elbow_angle_y_biomech", "elbow_angle_z_biomech"]
    extremity_index = joint_angle_extremity_index(granular_data, joint_cols=joint_cols, percentile_threshold=90)
    rom_progression = rom_progression_metric(granular_data, joint_cols=joint_cols, window_size=60)
    
    joint_muscle_map = {
        "FDS": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"],
        "FCU": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"],
        "FCR": ["elbow_angle_x_biomech", "elbow_angle_y_biomech"]
    }
    rest_lengths = {"FDS": 1.0, "FCU": 1.0, "FCR": 1.0}
    overextension_data = muscle_overextension_index(granular_data, joint_muscle_map, rest_lengths, critical_threshold=1.2)
    
    elbow_joint_col = "elbow_angle_z_biomech"
    elbow_velocity_col = "elbow_velo_z_biomech"
    granular_data[elbow_velocity_col] = np.random.uniform(50, 100, num_samples)
    velocity_data = velocity_at_extreme_rom(granular_data, joint_cols=[elbow_joint_col], velocity_cols=[elbow_velocity_col], percentile_threshold=90)
    
    def injury_indicator_elbow(kinematic_data, elbow_angle_col="elbow_angle_z_biomech", threshold=120):
        return (kinematic_data[elbow_angle_col] > threshold).astype(int)
    
    granular_data["elbow_injury_flag"] = injury_indicator_elbow(granular_data, elbow_angle_col="elbow_angle_z_biomech", threshold=120)
    
    def injury_indicator_muscle(exhaustion_index, threshold=1.0):
        return (exhaustion_index > threshold).astype(int)
    
    granular_data["muscle_injury_flag"] = injury_indicator_muscle(exhaustion_index, threshold=1.0)
    
    print("\nComposite Exhaustion Index (first few values):")
    print(exhaustion_index.head())
    print("\nMuscle Contraction Score (first few values):")
    print(contraction_score_data.head())
    print("\nExhaustion Rate from New Metrics (first few values):")
    print(exhaustion_rate_data.head())
    print("\nWavelet Features (first few rows):")
    print(wavelet_features.head())
    print("\nJoint Angle Extremity Index (first few values):")
    print(extremity_index.head())
    print("\nROM Progression Metric (first few values):")
    print(rom_progression.head())
    print("\nMuscle Overextension Data (first few values):")
    print(overextension_data.head())
    print("\nVelocity at Extreme ROM for Elbow (first few values):")
    print(velocity_data.head())
    print("\nElbow Injury Flag (distribution):")
    print(granular_data["elbow_injury_flag"].value_counts())
    print("\nMuscle Injury Flag (distribution):")
    print(granular_data["muscle_injury_flag"].value_counts())
    
    test_pitcher_id = granular_data["pitcher_id"].iloc[0]
    fatigue_units_improved = improved_fatigue_units(
        granular_data,
        exhaustion_index=exhaustion_index,
        extremity_index=extremity_index,
        force_col="valgus_torque_biomech",
        time_interval_col="ongoing_timestamp_biomech",
        default_interval=15,
        recovery_factor=0.1,
        velocity_col="Pitch Velocity (mph)",
        pitcher_id=test_pitcher_id,
        all_pitch_data=granular_data,
        weights=[0.4, 0.3, 0.3]
    )
    granular_data["fatigue_units_improved"] = fatigue_units_improved
    
    fatigue_units_original = composite_fatigue_units(
        granular_data,
        exhaustion_index=exhaustion_index,
        extremity_index=extremity_index,
        force_col="valgus_torque_biomech",
        time_interval_col="ongoing_timestamp_biomech",
        default_interval=15,
        recovery_factor=0.1,
        velocity_col="Pitch Velocity (mph)",
        baseline_velocity=92.16,
        weights=[0.4, 0.3, 0.3]
    )
    granular_data["fatigue_units_original"] = fatigue_units_original
    granular_data["fatigue_injury_flag"] = injury_indicator_fatigue_units_percentile(fatigue_units_improved, percentile=95)
    
    # -----------------------------
    # Compute Overall ACWR from improved fatigue units
    # -----------------------------
    overall_acwr = compute_acwr(granular_data["fatigue_units_improved"], acute_window=7, chronic_window=28)
    granular_data["ACWR_overall"] = overall_acwr
    print("\nOverall Acute-to-Chronic Workload Ratio (ACWR) (first few values):")
    print(granular_data["ACWR_overall"].head())
    
    # -----------------------------
    # Compute Per-Joint ACWR using ROM progression metric
    # -----------------------------
    # Filter columns in rom_progression that include '_rom_progression'
    acwr_per_joint = compute_acwr_df(rom_progression.filter(like="_rom_progression"), acute_window=7, chronic_window=28)
    print("\nPer-Joint ACWR (first few rows):")
    print(acwr_per_joint.head())
    
    # -----------------------------
    # Compute Per-Muscle ACWR using activation decay index (columns ending with '_decay_rate')
    # -----------------------------
    acwr_per_muscle = compute_acwr_df(decay_data.filter(regex="_decay_rate$"), acute_window=7, chronic_window=28)
    print("\nPer-Muscle ACWR (first few rows):")
    print(acwr_per_muscle.head())
    
    progression_features = create_progression_features(pd.DataFrame({"fatigue_units_improved": fatigue_units_improved}))
    print("\nImproved Cumulative Fatigue Units (first few values):")
    print(fatigue_units_improved.head())
    print("\nOriginal Cumulative Fatigue Units (first few values):")
    print(fatigue_units_original.head())
    print("\nFatigue Injury Flag (distribution):")
    print(granular_data["fatigue_injury_flag"].value_counts())
    print("\nProgression Features for Fatigue Units (first few rows):")
    print(progression_features.head())


   elbow_angle_x_biomech       arm_slot
0              89.063609  three-quarter
1              28.418830      submarine
2              41.599916        sidearm
3              44.867660        sidearm
4              40.133052        sidearm
[DEBUG] Muscle EMG 1 (mV) - FDS (81770) | Raw data summary:
count    100.000000
mean       1.170494
std        0.433102
min        0.502590
25%        0.757739
50%        1.115534
75%        1.516776
max        1.964563
Name: EMG 1 (mV) - FDS (81770), dtype: float64
[DEBUG] Muscle EMG 1 (mV) - FDS (81770) | Initial activation (first 60 values): 1.1623
[DEBUG] Muscle EMG 1 (mV) - FDS (81770) | Rolling activation (first 10 values):
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: EMG 1 (mV) - FDS (81770), dtype: float64
[DEBUG] Muscle EMG 1 (mV) - FDS (81770) | Normalized activation stats:
count    71.000000
mean      0.997414
std       0.051507
min       0.914409
25%       0.957747
50%       0.978279
75%       1.04

c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
c:\Users\GeoffreyHadfield\Anaconda3\envs\data_science_fatigue_analysis\lib\site-packages\pywt\_multileve

# Utilizing Fatigue Units to Predict Injury Likelihood in Baseball Pitchers

Fatigue has long been identified as one of the most significant risk factors for pitching injuries, with research indicating that pitchers who regularly throw while fatigued have dramatically increased injury risk. The development of Fatigue Units as a workload monitoring metric represents a significant advancement in our ability to quantify, track, and ultimately predict injury risk in pitchers. This comprehensive report examines how Fatigue Units can be implemented as a predictive tool for pitcher injury prevention.

## The Development and Physiological Basis of Fatigue Units

Fatigue Units were developed in 2014 by Dr. Michael Sonne based on biomechanical and exercise physiology principles. Unlike traditional metrics such as simple pitch counts or innings pitched, Fatigue Units were specifically designed to represent what's happening physiologically in a pitcher's arm during activity[1]. This approach was created in response to research showing that not all pitches create equal stress on the arm, and not all innings represent equivalent workloads.

The physiological foundation of Fatigue Units lies in understanding muscle fatigue as a specific process that decreases force-generating capacity. Dr. Sonne and colleagues created a model that predicts how much force-generating capacity muscles lose during repeated activity[9]. This is particularly relevant for pitchers because muscle fatigue significantly impacts injury risk. As explained by Dr. Jeremy Bruce and Dr. James Andrews, fatigue of the flexor-pronator muscles in the forearm compromises joint stability and increases stress on the ulnar collateral ligament (UCL)[1]. Dr. Glenn Fleisig's research further demonstrated that muscle fatigue leads to joint laxity, which increases strain on the ligaments during throwing[1].

The consequences of muscular fatigue extend beyond just reduced performance. Multiple studies have documented that fatigue leads to measurable changes in pitching mechanics. Fatigued pitchers show significantly less maximum shoulder external rotation and knee flexion at ball release, along with reduced shoulder and elbow proximal force[4]. Research by Escamilla and colleagues found that fatigued pitchers adopt more upright trunk positions[4], while other studies have shown earlier hip movement in the pitch cycle, suggesting core muscle fatigue[6].

## How Fatigue Units Calculate and Measure Workload

What makes Fatigue Units revolutionary is their incorporation of recovery time between throws—a critical factor traditional pitch counts ignore. While the average time between pitches in a game is approximately 15 seconds, the typical time between throws during long toss and bullpen warmups is just 3.5 seconds[3]. This difference significantly impacts how muscles recover from fatigue, yet has been largely overlooked in traditional workload measurements.

Fatigue Units measure accumulated workload by considering:

1. Peak torque of each individual throw
2. The time interval between consecutive throws
3. The physiological recovery characteristics of the involved muscles

The calculation results in a cumulative measure that increases during periods of high-frequency throwing and regenerates during rest periods[3]. This dynamic measurement provides a more accurate representation of the physiological stress a pitcher experiences than static pitch counts.

When implemented in tools like the motusDASH web application, Fatigue Units can be computed over each second of a throwing day, allowing for precise monitoring of acute fatigue development[3]. This granular measurement capability makes it particularly useful for identifying potentially dangerous patterns that might be missed by cruder metrics.

## Predictive Value Compared to Traditional Metrics

Traditional workload metrics like innings pitched have been shown to be poor determinants of actual stress on a pitcher's arm[1]. Research by Karakolis and colleagues confirmed that pitch counts or innings pitched may not effectively measure true workload[8]. The limitations of these traditional measures become evident when considering that not all innings create equal stress—throwing 100 pitches over a complete game is likely less stressful than throwing the same number across just five innings[8].

Fatigue Units provide superior predictive value compared to these traditional metrics for several reasons:

1. They account for varying intensity levels between different types of throws
2. They incorporate recovery time, recognizing that muscle fatigue is a dynamic process
3. They align more closely with the physiological processes that actually lead to injury

The improved predictive capability has been demonstrated in multiple studies examining historical injury data. For example, analysis using Fatigue Units has been able to identify potentially dangerous warmup protocols that traditional metrics would miss entirely[3]. This is particularly important given that much of a pitcher's workload occurs outside of game situations in bullpen sessions, long toss, and warmups that go completely unmonitored in traditional approaches.

## Applications for Injury Prediction and Prevention

The practical application of Fatigue Units for injury prediction involves several key approaches:

### Acute:Chronic Workload Ratio Integration

Fatigue Units can be used in conjunction with Acute:Chronic Workload Ratio (ACWR) calculations to identify dangerous training patterns. The motusDASH platform has implemented a modified 9-day ACWR alongside Fatigue Units to identify both acute (within-day) fatigue and chronic fatigue accumulated over weeks[3][11]. This dual approach provides a more comprehensive picture of a pitcher's fatigue status and injury risk.

An analysis of injury-tagged datasets has shown that during periods preceding injuries, pitchers often displayed elevated ACWR alongside high Fatigue Units[11]. Conversely, in datasets where the ACWR was low (<1.0) and Fatigue Units were also low, injury risk was significantly reduced[11].

### Real-Time Monitoring and Intervention

One of the most valuable applications of Fatigue Units is their ability to enable real-time monitoring during training sessions. By tracking Fatigue Units as they accumulate, coaches and medical staff can:

1. Establish individualized thresholds for safe training
2. Identify when a pitcher needs to be removed from activity before injury occurs
3. Design optimized recovery protocols based on accumulated fatigue
4. Make data-informed decisions about pitch limits and rest periods

Research has shown that self-reported "throwing while fatigued" resulted in a 36x increase in injury risk[10]. With Fatigue Units, this subjective measure can be replaced with objective data, allowing for intervention before the pitcher reaches dangerous fatigue levels.

### Evaluating Training and Rehabilitation Programs

Fatigue Units have proven valuable for evaluating the safety and effectiveness of throwing programs, particularly rehabilitation protocols. By applying Fatigue Units calculations to established rehabilitation programs from institutions like the Andrews Institute, Texas Medical Institute, and Mayo Clinic, researchers have identified potential improvements to these protocols[11].

An "optimized return-to-throwing program" has been proposed that utilizes Fatigue Units to ensure appropriate chronic workload progression while minimizing potentially dangerous fluctuations in acute workload[11]. This approach represents a significant advancement over current standards that rely primarily on pitch counts and subjective assessments.

## Research Evidence Supporting Predictive Value

Multiple studies have demonstrated the predictive value of Fatigue Units for injury risk:

The implementation of pitch clocks in baseball has provided a natural experiment for testing Fatigue Units predictions. Dr. Sonne's model showed that pitch clocks could significantly increase muscle fatigue in pitchers by forcing faster repetition with less recovery time. His research demonstrated that peak fatigue levels are more than 18% higher when the pitch clock is implemented[9]. This aligns with anecdotal evidence of increased pitcher injuries following the implementation of pitch clocks in Major League Baseball[7].

A longitudinal study examined Little League World Series (LLWS) pitchers ten years after they competed. Among those who exceeded recommended pitch counts during the LLWS and went on to play professionally, 50% eventually required Tommy John surgery. In contrast, only 1.7% of those who stayed within pitch count recommendations needed the same surgery[6]. While this study used pitch counts rather than Fatigue Units specifically, it demonstrates the powerful predictive value of workload monitoring for long-term injury outcomes.

Dr. Sonne's research on fatigue modeling for the 100-pitch limit further demonstrated how even the most commonly used pitch count threshold in baseball fails to account for the variability in how those pitches are thrown. Two pitchers could both throw 100 pitches, but experience vastly different fatigue levels based on the distribution of those pitches and rest intervals[8]. Fatigue Units can quantify these differences.

## Practical Implementation of Fatigue Units in Pitcher Management

Implementing Fatigue Units for injury prediction requires specific approaches:

### Establishing Individual Baselines and Thresholds

Each pitcher has unique recovery capabilities and fatigue thresholds. Effective implementation of Fatigue Units requires:

1. Collecting baseline data during controlled throwing sessions
2. Identifying individual fatigue thresholds through performance monitoring
3. Establishing personalized "red zones" for accumulated Fatigue Units
4. Creating periodized training programs that balance fatigue accumulation with recovery

### Monitoring All Throwing Activities

A critical insight from Fatigue Units research is that all throwing activities contribute to cumulative stress—not just in-game pitches. A comprehensive monitoring program must track:

1. Warmup throws (which often occur at dangerously short rest intervals)
2. Bullpen sessions
3. Long toss and flat ground work
4. In-game pitches
5. Between-inning warmup throws (which are typically uncounted)

Research has shown that "the volume and intensity of throws during warm-up, plyocare, long toss, bullpen, flat grounds, and even pitches between innings are neglected in current workload standards"[10]. Fatigue Units allow for the integration of all these activities into a single cumulative measure.

### Adjusting Training Based on Fatigue Status

Effective use of Fatigue Units requires responsive training adjustments. When elevated Fatigue Units are detected, interventions might include:

1. Extended rest periods between throwing sessions
2. Modified throwing mechanics drills to reinforce proper form
3. Targeted strength training for the most fatigued muscle groups
4. Adjustment of throwing intensity rather than volume in subsequent sessions

## Conclusion

Fatigue Units represent a significant advancement in our ability to predict and prevent pitching injuries. By incorporating physiological principles of muscle fatigue, recovery dynamics, and biomechanical stress, they provide a more accurate representation of a pitcher's true injury risk than traditional metrics like pitch counts or innings pitched.

The evidence demonstrates that Fatigue Units can effectively identify dangerous workload patterns before injury occurs. Implementation in both professional and youth baseball could potentially reduce the alarming rate of serious arm injuries, particularly those requiring surgery like UCL reconstruction.

As wearable technology becomes more accessible and data analysis more sophisticated, Fatigue Units will likely play an increasingly central role in pitcher management across all levels of baseball. The most effective injury prevention programs will combine Fatigue Units monitoring with proper mechanics training, appropriate physical conditioning, and individualized recovery protocols to keep pitchers healthy and performing at their best.

Citations:
[1] https://fantasy.fangraphs.com/an-introduction-to-fatigue-units-a-new-method-for-evaluating-workloads/
[2] https://www.drivelinebaseball.com/2025/02/understanding-and-managing-training-fatigue-from-theory-to-practice/
[3] https://www.drivelinebaseball.com/2019/09/motus-advances-workload-models-with-fatigue-units/
[4] https://pmc.ncbi.nlm.nih.gov/articles/PMC6673423/
[5] https://uspbl.com/wp-content/uploads/2017/12/predicting-injury-in-professional-baseball-pitchers-from-delivery-mechanics-a-statistical-model-using-quantitative-video-analysis.pdf
[6] https://www.abca.org/magazine/magazine/2020-5-September_October/The_Change_Up_Limit_Pitch_Counts_to_Protect_Young_Players.aspx
[7] https://www.reddit.com/r/Torontobluejays/comments/1387wng/is_the_pitch_clock_causing_more_pitcher_injuries/
[8] http://www.mikesonne.ca/baseball/fatigue-inferences-on-a-100-pitch-limit/
[9] http://www.mikesonne.ca/baseball/why-pitch-clocks-are-the-devil/
[10] https://pmc.ncbi.nlm.nih.gov/articles/PMC7534929/
[11] https://www.researchgate.net/publication/336346055_EVALUATING_REHABILITATION_THROWING_PROGRAMS_INJURY_DATASETS_WITH_ACUTECHRONIC_RATIOS_AND_PHYSIOLOGY-BASED_FATIGUE_UNITS
[12] https://www.linkedin.com/pulse/understanding-arm-injuries-optimizing-recovery-using-gering-pt-dpt-qsxdc
[13] https://www.sportsbusinessjournal.com/Daily/Issues/2022/05/18/Technology/an-mlb-pitch-clock-is-imminent-and-maybe-only-tech-and-data-can-prevent-the-downside-pitcher-fatigue/

---
Answer from Perplexity: pplx.ai/share




# Automated Fatigue Thresholds for Pitchers: Concepts, Implementation, and Optimization

The development of automated fatigue thresholds for baseball pitchers represents a significant advancement in injury prevention technology. By leveraging biomechanical data and physiological principles, we can create more sophisticated monitoring systems that detect fatigue before injuries occur. This report examines the conceptual framework, implementation strategies, and potential improvements for automated fatigue threshold systems.

## Conceptual Foundation of Pitcher Fatigue Metrics

The foundation of effective fatigue monitoring begins with understanding the physiological mechanisms that connect fatigue to injury risk. Traditional pitch count metrics fail to capture the complete picture of arm stress, as research shows that not all pitches create equal stress on a pitcher's arm.

### Fatigue Units as a Fundamental Metric

Fatigue Units (FUs) were developed in 2014 by Dr. Michael Sonne to represent what's happening physiologically in a pitcher's arm during activity. Unlike traditional metrics such as simple pitch counts or innings pitched, FUs were specifically designed to account for the varying stress levels experienced by the throwing arm[1]. This approach responds directly to research showing significant limitations in conventional workload measures.

The calculation of Fatigue Units incorporates three critical elements:
1. Peak torque of each individual throw
2. The time interval between consecutive throws
3. The physiological recovery characteristics of the involved muscles

What makes this metric particularly valuable is its incorporation of recovery time between throws—a critical factor traditional pitch counts ignore. Research shows that the average time between pitches in a game is approximately 15 seconds, while the typical time between throws during warmups is just 3.5 seconds[1]. This difference significantly impacts how muscles recover from fatigue, yet has been largely overlooked in traditional workload measurements.

### Physiological Basis for Automated Thresholds

The physiological foundation for fatigue thresholds is well-established. Dr. Glenn Fleisig's research demonstrated that muscle fatigue leads to joint laxity, which increases strain on the ligaments during throwing[4]. Similarly, Dr. Jeremy Bruce and Dr. James Andrews explained that fatigue of the flexor-pronator muscles in the forearm compromises joint stability and increases stress on the ulnar collateral ligament (UCL)[4].

Research confirms that regularly pitching while fatigued has been associated with 36 times greater odds of injury requiring surgery[3]. This dramatic increase in risk highlights the critical importance of detecting fatigue before it reaches dangerous levels.

## Implementation of Automated Fatigue Thresholds

Creating effective automated thresholds requires a combination of appropriate metrics, data collection methods, and computational approaches.

### Key Metrics for Comprehensive Fatigue Monitoring

A robust fatigue monitoring system should incorporate multiple complementary metrics:

#### 1. EMG-Based Fatigue Indicators

The Composite Exhaustion Index combines multiple EMG-derived metrics including:
- Activation decay rates in key muscles
- Frequency shifts in muscle activation patterns
- Force-activation efficiency metrics

This composite measure provides a physiologically-grounded assessment of muscle fatigue. Values in sample data range from -0.314464 to -1.866018, with more negative values potentially indicating greater fatigue after inversion calculations have been applied[2].

#### 2. Biomechanical Load Metrics

Joint Angle Extremity Index quantifies the percentage of time joints spend at extreme ranges of motion. Sample data shows a composite value of 20.0 across measured elbow angles, indicating that joints are spending 20% of their time at extreme positions[2].

ROM Progression Metric tracks subtle changes in maximum joint angles over time. Values ranging from 0.000000 to 0.021091 for elbow angle x-axis can indicate progressive mechanical changes that may signal fatigue[2].

#### 3. Fatigue Units Index

The fatigue_units_index() function provided in the code calculates cumulative Fatigue Units based on:
```python
def fatigue_units_index(granular_data, force_col="Force (N)", time_interval_col=None, default_interval=15, recovery_factor=0.1):
    forces = granular_data[force_col]
    # Normalize force values to get a relative peak torque measure
    max_force = forces.max()
    normalized_forces = forces / max_force
    
    # Get time intervals
    if time_interval_col and time_interval_col in granular_data.columns:
        intervals = granular_data[time_interval_col]
    else:
        intervals = pd.Series(default_interval, index=granular_data.index)
    
    # Calculate fatigue contribution for each throw
    fatigue_contrib = normalized_forces * (default_interval / intervals)
    
    # Compute cumulative fatigue
    cumulative_fatigue = fatigue_contrib.cumsum() - recovery_factor * np.arange(len(granular_data))
    
    return cumulative_fatigue
```

This approach normalizes force values to estimate relative peak torque and accounts for recovery time between pitches[2].

### Threshold Determination Methods

Determining appropriate fatigue thresholds should follow one of several approaches:

#### 1. Individual Baseline Calibration

Each pitcher has unique recovery capabilities and fatigue thresholds. Effective implementation requires:
- Collecting baseline data during controlled throwing sessions
- Identifying individual fatigue thresholds through performance monitoring
- Establishing personalized "red zones" for accumulated Fatigue Units
- Creating periodized training programs that balance fatigue accumulation with recovery[1]

#### 2. Machine Learning-Based Classification

Advanced threshold determination can leverage machine learning approaches:
- LSTMs for capturing long-range dependencies in fatigue development
- Ensemble methods combining multiple algorithms for robust prediction
- Classification accuracy between 78% and 96% has been demonstrated[14]

For implementing an injury risk threshold based on Fatigue Units, the code includes:
```python
def injury_indicator_fatigue_units(fatigue_units, threshold):
    """
    Returns a binary flag (1 = injury risk) if cumulative fatigue units exceed the given threshold.
    """
    return (fatigue_units > threshold).astype(int)
```

#### 3. Acute:Chronic Workload Ratio Integration

Fatigue Units can be combined with Acute:Chronic Workload Ratio (ACWR) calculations:
- The motusDASH platform has implemented a modified 9-day ACWR alongside Fatigue Units
- This approach identifies both acute (within-day) fatigue and chronic fatigue accumulated over weeks
- Research has shown that periods preceding injuries often featured elevated ACWR alongside high Fatigue Units[1]

## Optimizing Automated Fatigue Thresholds

Several improvements can enhance the effectiveness of automated fatigue thresholds:

### Comprehensive Data Collection

A critical insight from Fatigue Units research is that all throwing activities contribute to cumulative stress—not just in-game pitches. A comprehensive monitoring system must track:
1. Warmup throws (which often occur at dangerously short rest intervals)
2. Bullpen sessions
3. Long toss and flat ground work
4. In-game pitches
5. Between-inning warmup throws (which are typically uncounted)[1]

Research has confirmed that "the volume and intensity of throws during warm-up, plyocare, long toss, bullpen, flat grounds, and even pitches between innings are neglected in current workload standards"[1]. Fatigue Units allow for the integration of all these activities into a single cumulative measure.

### Multi-Resolution Temporal Framework

Implementing a multi-resolution approach captures fatigue at different temporal scales:
1. Pitch-by-pitch sequence - capturing acute fatigue during a single session
2. Session-by-session sequence - tracking recovery between consecutive pitching sessions
3. Season-long sequence - identifying cumulative fatigue patterns[2]

This nested approach ensures that both immediate fatigue and long-term accumulation are properly monitored.

### Personalized Fatigue Profiles

Not all pitchers fatigue in the same way or at the same rate. Advanced systems should implement:
- Clustering pitchers by fatigue response patterns
- Customized warning thresholds based on individual risk factors
- Pitcher-specific recovery models calibrated to individual recovery rates

Research shows significant individual variation in how pitchers respond to fatigue. A study of adolescent pitchers found that while they generally became more fatigued as pitch count increased, the specific mechanical changes varied considerably between pitchers[2].

### Velocity-Adjusted Fatigue Units

Adding velocity as a scaling factor improves the accuracy of fatigue predictions:
- Create a scaling factor using average velocity (e.g., 92.16 mph baseline)
- Multiply the FU by this scaling factor to account for higher stress from higher velocity pitches
- Research shows that peak fatigue levels can be more than 18% higher when accounting for velocity factors[9]

## Conclusion

Creating effective automated fatigue thresholds for pitchers requires a sophisticated combination of physiological understanding, comprehensive data collection, and advanced computational methods. The most effective approach integrates Fatigue Units with biomechanical metrics like the Composite Exhaustion Index and Joint Angle Extremity Index, while accounting for individual differences between pitchers.

By implementing the improvements outlined above—particularly comprehensive data collection across all throwing activities and personalized baseline calibration—we can develop fatigue monitoring systems that accurately identify dangerous workload patterns before injury occurs. As wearable technology becomes more accessible and data analysis more sophisticated, these automated fatigue thresholds will play an increasingly central role in pitcher management across all levels of baseball, potentially reducing the alarming rate of serious arm injuries.

The ultimate goal of these systems should be to provide real-time monitoring during training sessions, forecast future fatigue states, generate adaptive recommendations for workload management, and optimize recovery protocols based on accumulated fatigue levels. With proper implementation, automated fatigue thresholds can revolutionize how pitchers are managed throughout a season, potentially reducing injuries while maximizing performance.

Citations:
[1] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/53658542/ef169ea9-b1ea-4468-828b-c7ea06825b43/paste.txt
[2] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/53658542/b41253d9-2f0c-4c56-b31e-316eceb25e58/paste-2.txt
[3] https://pmc.ncbi.nlm.nih.gov/articles/PMC7534929/
[4] https://fantasy.fangraphs.com/an-introduction-to-fatigue-units-a-new-method-for-evaluating-workloads/
[5] https://pmc.ncbi.nlm.nih.gov/articles/PMC8822888/
[6] https://pmc.ncbi.nlm.nih.gov/articles/PMC8711585/
[7] https://www.mdpi.com/1424-8220/19/17/3637
[8] https://www.reddit.com/r/baseball/comments/1eswgoz/why_a_sixinning_minimum_for_starting_pitchers/
[9] http://www.mikesonne.ca/tag/fatigue/
[10] https://www.researchgate.net/figure/Receiver-operating-characteristic-ROC-analysis-for-evaluating-the-potential-of-the-used_fig2_303689942
[11] https://www.drivelinebaseball.com/product/blast-motion-bat-sensor/
[12] https://baseballwithr.wordpress.com/2022/10/20/studying-pitcher-fatigue-using-a-multinomial-regression-model/
[13] https://pmc.ncbi.nlm.nih.gov/articles/PMC1896072/
[14] https://www.rrd.nl/wp-content/uploads/2023/02/59-Thesis-Luca-Marotta.pdf
[15] https://www.diamondkinetics.com/shop
[16] https://pmc.ncbi.nlm.nih.gov/articles/PMC9406287/
[17] https://www.target.com/c/pitchers-serveware-kitchen-dining/threshold/-/N-5xtqfZ56dig
[18] https://www.equippers.com/60-oz-clear-poly-beverage-pitcher
[19] https://nikojune.com/products/soft-serve-pitcher
[20] https://pubmed.ncbi.nlm.nih.gov/20231754/
[21] https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0045755
[22] https://www.researchgate.net/publication/326049920_Effect_of_Fatigue_on_Medial_Elbow_Torque_in_Baseball_Pitchers_A_Simulated_Game_Analysis
[23] https://www.drivelinebaseball.com/2021/08/how-the-cincinnati-reds-used-pulse-to-more-effectively-manage-their-pitching-staff/
[24] https://www.researchgate.net/figure/ROC-curve-of-the-resting-motor-threshold-considering-the-whole-of-population-p-0001_fig2_319645984
[25] https://www.drivelinebaseball.com/2025/02/understanding-and-managing-training-fatigue-from-theory-to-practice/
[26] https://www.mdpi.com/2078-2489/7/3/52
[27] https://baseballdevelopmentgroup.com/2019/01/fatigue-units-a-conversation-with-bdg-research-director-dr-mike-sonne/
[28] https://worksheets.codalab.org/rest/bundles/0xadf98bb30a99476ab56ebff3e462d4fa/contents/blob/glove.6B.100d.txt-vocab.txt
[29] https://blastmotion.com/products/baseball/
[30] https://www.reddit.com/r/baseball/comments/1ani08u/how_the_automated_balls_and_strike_system_abs/
[31] https://www.researchgate.net/publication/352422385_Prediction_of_athlete_movements_using_wearable_sensors_for_sports_person_health_monitoring_application
[32] https://www.researchgate.net/publication/358772328_Research_on_a_Real-Time_Driver_Fatigue_Detection_Algorithm_Based_on_Facial_Video_Sequences
[33] https://www.researchgate.net/publication/377026170_A_Real-Time_Automated_Defect_Detection_System_for_Ceramic_Pieces_Manufacturing_Process_Based_on_Computer_Vision_with_Deep_Learning
[34] https://www.linkedin.com/advice/3/how-can-you-use-computer-vision-track-human-w4nbe
[35] https://openreview.net/pdf/da00877e5148287d08c4fa6b41bf94356fa47e6e.pdf
[36] https://www.diamondkinetics.com/technology
[37] https://pmc.ncbi.nlm.nih.gov/articles/PMC9619001/
[38] https://pubmed.ncbi.nlm.nih.gov/36325144/
[39] https://www.mdpi.com/1424-8220/21/10/3451
[40] https://arxiv.org/html/2407.08081v1
[41] https://sabr.org/analytics/presentations/2021

---
Answer from Perplexity: pplx.ai/share




I'll analyze the best y variables for forecasting fatigue in time series per joint and muscle, as well as for injury classification. Additionally, I'll determine the best x features that can be engineered using both EMG and biomechanics data. This will help in predicting energy levels at each joint and muscle using an LSTM regression model and forecasting injury-prone moments using a time series classifier. I'll get back to you with detailed findings soon.

# Analysis of EMG and Biomechanics Data for Fatigue and Injury Prediction

## Introduction  
This report examines a dataset of electromyography (EMG) and biomechanical measurements to identify key factors for fatigue forecasting and injury risk prediction. The analysis addresses five main objectives: determining the best output variables (Y) for **fatigue forecasting** and for **injury classification**, proposing effective **feature engineering** techniques for input variables (X), and outlining modeling approaches using **LSTM regression** for fatigue and **time-series classification** for injury prediction. Each section below presents findings with supporting rationale, and tables are provided to summarize recommended variables and features.

## 1. Best Output Variables for Fatigue Forecasting  
**Defining Fatigue Measures:** To predict fatigue levels for each joint and muscle, the output variable (target) should quantify the degree of muscle exhaustion or performance decline over time. Effective fatigue indicators typically capture changes in muscle activation patterns, energy output, or stability of contractions. Table 1 summarizes recommended fatigue metrics (Y variables) per muscle/joint and why they are indicative of fatigue progression.

**Table 1. Candidate Fatigue Outcome Variables (Y) and Rationale**  

| **Fatigue Metric**                    | **Description**                                                | **Rationale**                                                |
|---------------------------------------|----------------------------------------------------------------|--------------------------------------------------------------|
| **EMG Median Frequency (MDF) Drop**   | Decline in the median frequency of the muscle’s EMG power spectrum over time. Often quantified as the slope or percent change of MDF during sustained activity. | MDF decreases roughly linearly with increasing muscle fatigue ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)) as conduction velocity slows. A faster drop in MDF indicates quicker onset of fatigue ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)). This spectral shift reflects the muscle’s inability to maintain high-frequency contractions. |
| **EMG Amplitude Rise (RMS/iEMG)**     | Increase in EMG signal amplitude (e.g. Root Mean Square or integrated EMG) needed to maintain the same force output. Typically measured per muscle. | As muscles fatigue, they recruit more fibers or generate higher activation to compensate, causing EMG amplitude (RMS, iEMG) to rise over time ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=generally%20show%20an%20upward%20trend,Therefore%2C%20RMS)). A steady increase in RMS indicates growing effort and energy consumption, a sign of fatigue ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=generally%20show%20an%20upward%20trend,Therefore%2C%20RMS)). |
| **Muscle Fatigue Index (Spectral)**   | A composite fatigue index capturing spectral shifts (e.g. Dimitrov’s index or wavelet-based index *I<sub>CWT</sub>*). | Fatigue indices combine multiple spectral features to quantify fatigue more robustly than a single feature. For example, wavelet-based fatigue indices track EMG signal elongation and are less noise-sensitive than simple mean/median frequency ([
            EMG Processing Based Measures of Fatigue Assessment during Manual Lifting - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC5337807/#:~:text=Yochum%20et%20al,signal%20elongation%20during%20a%20contraction)). Such indices provide a numeric “fatigue level” per muscle. |
| **Joint Power Output Decline**        | Reduction in mechanical power or work done at a joint over time (e.g. decline in peak joint moment or angular velocity). | Fatigued muscles produce less force and slower movements, leading to lower joint power. A downward trend in peak joint moment or joint work indicates that the joint’s supporting muscles are tiring (exhaustion rate). This reflects diminished energy transfer through that joint as fatigue sets in. |
| **Contraction Stability Index**       | Increase in variability or co-contraction imbalance during repeated movements. For instance, the ratio of agonist to antagonist muscle activation (co-contraction) or variability in joint kinematics. | Fatigue often brings instability in muscle contractions and coordination. A growing agonist/antagonist imbalance (e.g. quadriceps vs hamstring activation ratio) or more variable joint angles can quantify loss of muscular control. Stable contraction patterns deteriorate as exhaustion approaches, so a stability index captures this decline. |

*Justification:* The above variables directly measure how muscles and joints behave as they tire. Spectral measures like median frequency are well-established fatigue markers (dropping as fatigue deepens) ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)), while time-domain amplitude measures rise with increased effort in fatigued states ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=generally%20show%20an%20upward%20trend,Therefore%2C%20RMS)). Combining multiple indicators (e.g. through a fatigue index) can improve robustness by capturing different aspects of fatigue (electrical and mechanical changes). Joint-level outputs (power or moment decline) tie muscle fatigue to functional performance loss at the joint, indicating the **energy transfer** efficiency drop. Finally, measures of **muscle contraction stability** (coordination consistency) address how exhaustion leads to erratic muscle/joint behavior, which is relevant for forecasting when a muscle or joint is nearing fatigue failure.

## 2. Best Output Variables for Injury Classification  
**Defining Injury-Risk Moments:** For injury prediction, the output of interest is a classification label indicating high injury risk or an impending injury event. In a dataset, this could be a binary label per time window (e.g. 1 = likely injury moment, 0 = normal). To determine this, we identify biomechanical and EMG variables that signal unsafe conditions. Key factors include abnormal joint loads (especially knee valgus/abduction moments), extreme angles, poor muscle coordination, or instability—especially under fatigue. Table 2 outlines the most telling variables related to injury risk and how they inform the classification.

**Table 2. Key Injury Risk Indicators and Their Significance**  

| **Injury Indicator**                    | **Description**                                                  | **Why It Indicates Injury Risk**                               |
|-----------------------------------------|------------------------------------------------------------------|----------------------------------------------------------------|
| **Knee Valgus Angle & Moment**          | Dynamic knee valgus angle (inward collapse of the knee) and external knee abduction moment (KAM, valgus torque) during movements like landing or cutting. | Excessive knee valgus and high abduction moment are strongly linked to ACL injuries ([Frontiers | Athletes with high knee abduction moments show increased vertical center of mass excursions and knee valgus angles across sport-specific fake-and-cut tasks of different complexities](https://www.frontiersin.org/journals/sports-and-active-living/articles/10.3389/fspor.2022.983889/full#:~:text=a%20biomechanical%20perspective%2C%20the%20external,rotation%20substantially%20increases%20ACL%20strain)). A high KAM shifts load to the lateral knee and combined with valgus/internal tibial rotation greatly increases ACL strain ([Frontiers | Athletes with high knee abduction moments show increased vertical center of mass excursions and knee valgus angles across sport-specific fake-and-cut tasks of different complexities](https://www.frontiersin.org/journals/sports-and-active-living/articles/10.3389/fspor.2022.983889/full#:~:text=a%20biomechanical%20perspective%2C%20the%20external,rotation%20substantially%20increases%20ACL%20strain)). Thus, peaks in valgus angle/moment serve as a direct flag for injury-prone mechanics. |
| **Joint Impact and Loading Rates**      | Peak ground reaction force and loading rate at foot strike or landing; sudden spikes in joint force or torque. | High impact forces or torque applied in a short time can exceed tissue capacity. A rapid loading (e.g. heavy heel strike or quick twist) suggests the joint is experiencing stress beyond normal, a scenario often preceding injuries (like ligament tears or cartilage damage). These features help classify moments of dangerous impact. |
| **Muscle Activation Imbalance**         | Poor coordination or timing between muscle groups (e.g. low hamstring activation relative to quadriceps during knee loading, quantified by H/Q EMG ratio or co-contraction index). | Imbalanced muscle support can let joints move into hazardous positions. For example, if quadriceps fire without sufficient hamstring co-activation, the anterior knee is less protected, elevating ACL injury risk. A sudden drop or disparity in a muscle’s activation when it *should* be active is a red flag for instability. |
| **Fatigue-Induced Instability**         | Signs of compromised stability due to fatigue: increased variability in joint kinematics (wobble) or delayed reflex responses. Often evaluated via variation in consecutive movement cycles or sway measures. | Fatigued muscles can no longer stabilize joints effectively ([
            Fatigue Effects on Knee Joint Stability During Two Jump Tasks in Women - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC2869370/#:~:text=Dynamic%20knee%20joint%20stability%20may,and%20hamstrings%2C%20and%20quadriceps%2Fhamstring%20EMG)). Studies show metabolic fatigue impairs dynamic joint stability, leading to larger valgus angles and riskier mechanics during landings ([
            Fatigue Effects on Knee Joint Stability During Two Jump Tasks in Women - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC2869370/#:~:text=During%20both%20sessions%2C%20peak%20knee,varus%2Fvalgus%20internal%20joint%20moments%20for)). Thus, a combination of high fatigue (from Section 1 metrics) *plus* abnormal motion variance can classify an “at-risk” moment. |
| **Joint Range Extremes**                | Joint angles approaching physiological limits (e.g. hyperextension of knee or elbow, excessive ankle inversion). | When a joint nears or exceeds its normal range under load, injury is likely (ligaments or capsules get overstretched). These extreme kinematic positions, especially if combined with high force, are clear indicators that the movement is unsafe. |

*Justification:* The above indicators cover both the **magnitude of risky loads** on the joints and the **neuromuscular control aspects** that typically precede injuries. Knee valgus/abduction moment stands out as a crucial variable, given its well-documented role in ACL injury mechanisms ([Frontiers | Athletes with high knee abduction moments show increased vertical center of mass excursions and knee valgus angles across sport-specific fake-and-cut tasks of different complexities](https://www.frontiersin.org/journals/sports-and-active-living/articles/10.3389/fspor.2022.983889/full#:~:text=a%20biomechanical%20perspective%2C%20the%20external,rotation%20substantially%20increases%20ACL%20strain)). Likewise, poor muscle coordination (e.g. inadequate hamstring braking) and instability from fatigue link the internal state of the athlete to injury propensity — fatigued or uncoordinated muscles allow dangerous joint motions ([
            Fatigue Effects on Knee Joint Stability During Two Jump Tasks in Women - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC2869370/#:~:text=During%20both%20sessions%2C%20peak%20knee,varus%2Fvalgus%20internal%20joint%20moments%20for)). By using these variables as classification targets or triggers (for labeling segments as “injury likely”), we ensure the model focuses on biomechanical situations that truly correlate with injury events. The classification output (Y) can thus be defined as a binary label derived from thresholds on one or a combination of these metrics (for example, label an *injury-risk* frame if KAM > X and co-contraction ratio below Y, etc.).

## 3. Feature Engineering for Input Variables (X)  
To improve predictive modeling, we derive informative features from the raw EMG and biomechanics time-series. Feature engineering transforms raw signals into higher-level representations that capture patterns related to fatigue and injury. We consider several categories of features, leveraging time-domain statistics, frequency-domain analysis, time-frequency transformations (e.g. wavelets), and trend features from rolling windows. Each technique extracts complementary information:

- **Time-Domain Features:** Simple statistical measures from the EMG or kinematic signals over a window. These include:  
  - *RMS (Root Mean Square) amplitude:* reflects muscle activation level ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=generally%20show%20an%20upward%20trend,Therefore%2C%20RMS)). Higher RMS can indicate greater effort or muscle recruitment.  
  - *Integrated EMG (iEMG):* the area under the EMG curve in a time window, correlating with total muscle effort.  
  - *Mean/Max Joint Angle or Velocity:* e.g. average knee flexion angle or peak angular velocity in a gait cycle, indicating movement patterns.  
  - *Zero-Crossing Rate (ZCR):* frequency of sign changes in EMG, related to muscle activation firing rate.  
  - *Co-Contraction Index:* difference or ratio of two muscles’ EMG (agonist vs antagonist) over time, capturing coordination.  
  These time-domain features are intuitive and fast to compute ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=Feature%20extraction%20from%20the%20sEMG,2020)) ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=generally%20show%20an%20upward%20trend,Therefore%2C%20RMS)). They often serve as baseline predictors (e.g. rising RMS signals fatigue, extreme joint angles signal risky movement).

- **Frequency-Domain Features:** Features computed from the power spectrum of signals, useful for capturing fatigue-related frequency shifts or characteristic frequencies of movement. Key examples:  
  - *Median and Mean Frequency of EMG:* as discussed, these decrease with fatigue due to muscle fiber conduction changes ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)). Including the current value or recent trend of median frequency is valuable for fatigue forecasting.  
  - *Spectral Entropy or Energy in Bands:* measures the distribution of power across frequencies. Fatigue can cause power to concentrate in lower frequencies (in EMG), which would be reflected in band-specific energy features or a change in entropy.  
  - *Peak Frequency of Motion:* for biomechanical signals like ground reaction force or joint angle, the dominant frequency might relate to cadence or oscillation – changes could indicate altered gait under fatigue or instability.  
  Frequency features add insight beyond amplitude, especially for EMG where fatigue manifests as frequency content shifts ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)).

- **Time-Frequency and Wavelet Features:** Using wavelet transforms or short-time Fourier transforms to capture how signal characteristics evolve over time. Techniques include:  
  - *Wavelet Coefficients & Energy:* Decompose EMG signals with wavelet transform to capture both time and frequency localization. For example, compute the energy in specific wavelet frequency bands over each movement or contraction. This can reveal transient muscle fatigue indicators that pure FFT might miss. Wavelet-based features have been shown to provide robust fatigue indices less sensitive to noise ([
            EMG Processing Based Measures of Fatigue Assessment during Manual Lifting - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC5337807/#:~:text=Yochum%20et%20al,signal%20elongation%20during%20a%20contraction)).  
  - *Wavelet Packet or Continuous Wavelet Fatigue Index:* As in research by Yochum et al., a continuous wavelet transform-based index *I<sub>CWT</sub>* can serve as an advanced feature for muscle fatigue ([
            EMG Processing Based Measures of Fatigue Assessment during Manual Lifting - PMC
        ](https://pmc.ncbi.nlm.nih.gov/articles/PMC5337807/#:~:text=Yochum%20et%20al,signal%20elongation%20during%20a%20contraction)). Including such indices or even the raw wavelet detail coefficients at certain scales could help the model detect subtle changes preceding fatigue or injury.  
  - *Time-Frequency Moments:* Statistics like the instantaneous median frequency (IMDF) which track frequency changes moment-to-moment ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=frequency%20%28MF%29%20%28Hou%20et%20al,practical%20application%2C%20MDF%20is%20more)), effectively embedding time-evolving spectral info as features.  

- **Rolling Window Trend Features:** These capture how signals change over short-term periods and can serve as precursors for forecasting:  
  - *Slope/Derivative:* The slope of a feature computed over a rolling window (e.g. rate of change of EMG median frequency or of joint angle). A steep negative slope in median frequency over recent seconds, for instance, indicates rapidly worsening fatigue – a strong predictor for imminent fatigue threshold.  
  - *Moving Averages:* Smooth the signal to observe general trend (e.g. a 5-second moving average of RMS EMG to capture gradual fatigue build-up, filtering out high-frequency noise).  
  - *Variance or Coefficient of Variation:* Within a window, the variability of a signal (e.g. variability of step-by-step knee angle). Increased variance in kinematics could be a feature flagging loss of consistency in motion (potential instability).  
  - *Peaks Count:* Number of peaks or abnormal spikes in a window (for forces or accelerations) – could indicate erratic behavior due to fatigue or lapses in form.  

- **Cross-Domain Features:** Combining EMG and biomechanics data to derive interaction features:  
  - *EMG-Force Delay:* Time lag between muscle activation burst and force production (electromechanical delay). Fatigue might lengthen this delay. Tracking it could improve injury risk prediction, as longer delays mean less dynamic stability.  
  - *Synergy or PCA Components:* Use techniques like principal component analysis to capture major patterns of multi-muscle activation. These components can quantify overall coordination. Changes in the synergy pattern (e.g. a new principal component emerging as fatigue sets in) are informative features.  
  - *Joint Work vs. EMG Effort:* The ratio of mechanical work done at a joint to the integrated EMG of prime movers – essentially an efficiency metric. Dropping efficiency (more EMG for same work) indicates fatigue and could correlate with higher injury risk (muscles struggling to produce required force).  

*Feature Selection:* After generating a rich set of features, one can perform dimensionality reduction or selection (e.g. via correlation analysis or PCA ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=match%20at%20L452%20feature%20extraction,2018a))) to keep the most predictive ones. Typically, a mix of time-domain and frequency-domain features yields the best performance for fatigue detection ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=Feature%20extraction%20from%20the%20sEMG,2020)). For injury classification, features capturing extremes or sudden changes (peaks, variance) are particularly useful since injuries are often associated with abrupt aberrations from normal patterns. It’s also beneficial to normalize features per subject (or muscle) to account for individual differences (e.g. EMG amplitude normalized to each subject’s max voluntary contraction). By engineering these features, we enhance the model’s ability to learn the underlying patterns of fatigue and injury from the raw EMG/biomechanical signals.

## 4. LSTM Regression for Fatigue Forecasting  
**Selecting Time-Series Variables:** Long Short-Term Memory (LSTM) networks are well-suited to model temporal dependencies in sequential data, making them a natural choice for fatigue forecasting. To train an LSTM regression model that predicts fatigue levels, we utilize time-series variables that evolve with fatigue. These include both raw signal sequences and the engineered features from Section 3 (especially those that are time-sequenced). Key inputs to consider are: 

- **Sequential EMG Features:** Instead of single-time-point features, we feed sequences of features such as the muscle’s RMS EMG, median frequency, or wavelet energy, sampled over time. For example, a sliding window of 5 seconds of EMG median frequency values can be an input sequence to predict the fatigue index at the end of that window. Including multiple muscles’ sequences (multi-variate input) allows the LSTM to learn joint fatigue trends (e.g. quadriceps and hamstring fatigue together for knee joint fatigue forecasting).  
- **Biomechanical Time-Series:** Sequences of joint kinematics or kinetics can also be inputs. For instance, the time series of knee moment and angle over each gait cycle can help forecast when performance is degrading. If fatigue manifests as gradually reduced knee flexion angle or slower velocity, the LSTM can capture that trajectory.  
- **Lagged Variables and Past Outputs:** The LSTM by design retains memory of past inputs, but one can also explicitly include lagged features. For example, append a feature that is the fatigue metric from the previous time step (t-1) or an exponential moving average of past fatigue level. This gives the model an anchor in recent history and can stabilize predictions. In essence, the LSTM’s hidden state will learn these lags, but including them as explicit input features (like a prior fatigue estimate or prior heart rate if available) can provide additional context.

**Window Size and Sequence Length:** Choosing the right sequence length (window size) for the LSTM is critical. We analyze different window sizes to find an optimal trade-off: 

- *Short Windows (e.g. 1–2 seconds or a few dozen samples):* These capture immediate muscle response but may miss the broader fatigue trend. A very short window might lead the LSTM to focus on high-frequency fluctuations rather than true fatigue progression, resulting in a noisier prediction.  
- *Long Windows (e.g. 30+ seconds or hundreds of samples):* These provide more history for the LSTM to detect slow fatigue trends, but if too long, the sequence may include irrelevant older data and make learning harder. Long sequences also increase training time and risk overfitting if the model memorizes particular long patterns.  
- *Moderate Windows (e.g. 5–10 seconds or one movement cycle):* Often a sweet spot, long enough to encompass a full muscle usage cycle (like a full repetition or stride) and some fatigue development, but short enough to stay focused. For instance, using a window that spans a complete lift or gait cycle ensures the LSTM sees the beginning-to-end behavior of muscles for that cycle, which is meaningful for predicting fatigue at the end of it.

In practice, one would experiment with different window lengths. Suppose we tried window lengths of 50, 100, and 200 timesteps (covering, say, 2s, 5s, and 10s respectively). We might find that the 5-second (100 timestep) window yields the lowest prediction error for fatigue level because it balances recent trend and manageable sequence length. Shorter 2-second windows might not capture enough of the fatigue trend (model had to guess based on very limited change), while 10-second windows did not significantly improve accuracy but added complexity. 

**Effectiveness of LSTM for Fatigue:** The memory cells in LSTMs allow the model to learn how fatigue accumulates over time. Prior studies have found LSTM models effective in recognizing fatigue patterns from sEMG sequences ([Frontiers | Application of Surface Electromyography in Exercise Fatigue: A Review](https://www.frontiersin.org/journals/systems-neuroscience/articles/10.3389/fnsys.2022.893275/full#:~:text=machine%20,MLPNN%29%20which)), outperforming static models since they utilize temporal context. By training on sequences of input features and the corresponding fatigue metric (e.g. fatigue index or endurance time), the LSTM regression can forecast fatigue level one step ahead (predicting the next timepoint’s fatigue) or even multi-step ahead if needed. Including appropriate lag features (or simply relying on the LSTM’s gating to carry memory) helps capture the *rate* of fatigue – for example, if the last few windows show a steep drop in median frequency, the model will extrapolate that the fatigue level is approaching a critical threshold soon.

**Training Considerations:** We would train the LSTM on many overlapping sequences extracted from the dataset. Each sequence (X) could correspond to a time window of sensor data, with the target (Y) being the fatigue measure at the end or slightly after that window (to truly *forecast* ahead). We ensure to shuffle or randomize sequences from different sessions to make the model generalize. Monitoring validation error for different sequence lengths can guide the optimal window choice. Additionally, we might use techniques like early stopping or regularization (dropout in LSTM layers) to prevent overfitting, especially if long sequences are used. 

In summary, **the best time-series variables for LSTM input will be those that evolve smoothly with fatigue (to leverage temporal patterns)** – primarily filtered EMG features and joint performance metrics – and the window size should be tuned such that it captures enough history to see the fatigue trend without diluting the relevance of older data. By examining multiple window sizes and including necessary lagged context, we can maximize the LSTM’s forecasting accuracy for fatigue.

## 5. Time-Series Classification for Injury Prediction  
**Dataset Structuring for Classification:** Injury prediction is framed as a time-series classification problem, where we classify segments of data as either *injury-risk* or *normal*. The first step is preparing the dataset appropriately:

- **Segmentation into Windows:** We segment the continuous data into time windows (sequences) that will serve as samples for classification. Each window could be a fixed length (e.g. a few seconds, or the duration of a movement like a jump or stride) and will be assigned a label. Windows during which an injury or near-injury event occurred (or where the injury risk indicators from Section 2 exceed safe limits) are labeled as **1 (injury likely)**, and others as **0 (no injury)**. For example, if the dataset contains a known injury at a specific time, we can label the window leading up to and including that moment as injury=1. If using proxies (like high valgus) for likely injury moments, any window where knee valgus angle and moment go beyond a threshold could be labeled 1 as well. Non-injury windows should represent typical, safe movement periods.

- **Window Length and Overlap:** The window length should be chosen to capture the full context of a potential injury event. Often, injuries in sports happen within a second or two (e.g. the landing phase of a jump). A window that covers a bit before and after the event (say 1–2 seconds total) can give the classifier both the lead-up and the instant of injury. We may use overlapping windows (sliding window approach) to increase the number of training samples – for instance, a new window starting every 0.5 seconds – which also ensures the injury event (a short instant) appears in multiple windows for better learning. Overlap also helps smooth the classification output over time when the model is used in practice.  

- **Balancing Classes:** Injury occurrences are typically much rarer than normal activities, leading to class imbalance. To address this, we ensure the training dataset is balanced or use techniques to compensate:
  - *Oversampling:* We can include the injury-likely windows multiple times or generate augmented versions of them (e.g. adding slight noise to signals or time-warping the sequence) to effectively increase the minority class count. 
  - *Undersampling:* Reduce the number of normal windows by sampling only as many as needed (or weighting them less in training) so that the classifier doesn’t become biased toward outputting “no injury” all the time.
  - *Class Weights:* In model training (especially for neural network classifiers), assign a higher weight to injury class examples so that misclassifying an injury carries a larger penalty. This helps the model pay sufficient attention to the injury class despite fewer examples.

- **Feature Extraction for Classification:** Just as with fatigue, we can feed the classifier either raw sequences or extracted features from each window (or a hybrid of both). There are two common approaches:
  1. *Manual Feature-Based Classification:* For each window, compute a set of features (like those in Section 3) that summarize that window’s characteristics. For example, features could include the peak knee valgus angle in the window, the average co-contraction index, the variance of joint angle, the EMG median frequency (to reflect fatigue state), etc. This reduces each window to a feature vector, and then a conventional classifier (like Random Forest, SVM, or an MLP) or even a time-series specialized classifier (like a shapelet-based classifier) can be trained on these feature vectors. The advantage is interpretability and lower-dimensional data. However, one must be careful to include all relevant dynamics in the features.
  2. *End-to-End Sequence Classification:* Feed the time-series window (multi-channel sequence) into a sequential model (like an LSTM or a 1D CNN) that learns to classify directly from the raw or lightly processed signals. For instance, an LSTM classifier could take in the sequence of sensor readings (EMG of multiple muscles, joint angles, etc. over time) and output a binary label. This approach can capture subtle temporal patterns (like the timing between muscle activations and impact) that might be lost in manual features. The MDPI study by Wang *et al.* is an example where an LSTM was used to classify muscle fatigue from sEMG sequences, using a moving window segmentation of 2 seconds ([A Muscle Fatigue Classification Model Based on LSTM and Improved Wavelet Packet Threshold](https://www.mdpi.com/1424-8220/21/19/6369#:~:text=sEMG%20signals%20by%20a%20moving,AT%20in%20the%20fatigue%20test)) – a similar concept can be applied for injury classification.

- **Enhancing Classification Accuracy:** Regardless of approach, certain feature and modeling techniques can improve accuracy:
  - *Inclusion of Fatigue State:* Because fatigue can precede injury, including a feature that represents the current fatigue level of key muscles (possibly computed by the fatigue forecasting model) could improve injury prediction. A model that knows the athlete is highly fatigued may interpret risky movements as more likely to cause injury.
  - *Temporal Order Features:* If using feature-based classification, consider features that capture dynamics, not just static values. For example, “change in knee angle in the last 100ms” or “difference between early-window and late-window muscle activation”. These mimic what an LSTM would learn about sequence shape but in explicit terms.
  - *Sliding Window Ensembles:* To handle sequences of varying possible lengths or uncertainties in timing, an ensemble of classifiers on different window sizes could be used. One classifier might look at a 0.5s window (catch very acute signatures), another at 1.5s window (broader context), and combine their outputs.
  - *Threshold Tuning:* When the goal is to predict “moments when an injury is likely,” it might be acceptable to tune the classifier for high sensitivity (catch all potential injury events) at the expense of some false alarms. This can be done by adjusting the decision threshold (if using a probabilistic classifier) or by incorporating the cost of false negatives in the training objective.

**Dataset Example Structure:** Suppose our dataset has time-series columns for several EMG channels (muscles) and biomechanical sensors (joint angles, moments, forces), and we have labeled a few time points where an injury (or high risk) occurred. We would create training sequences as follows: for each injury event at time *t*, take a window [t–Δ, t+Δ] around it (covering a bit before and after) as a positive sample. For negative samples, take many windows from times where everything was normal (making sure they don’t overlap with injury events). We ensure that each window is labeled and use these to train the model. If the injury events are extremely sparse, we might create multiple positive samples per event by shifting the window slightly (since the exact alignment might be uncertain).

**Evaluation:** We would typically evaluate the time-series classifier by its accuracy, precision-recall, or ROC curve in detecting injury-risk windows. Given the likely imbalance, metrics like F1-score or AUROC are more informative than raw accuracy. Cross-validation should be done in a way that prevents leakage (e.g. ensure that windows from the same continuous sequence are not split between train and test in a way that the model could get an unfair clue).

By structuring the data into labeled time windows and engineering features that highlight the known precursors to injury (valgus moments, instability, etc.), we set up the time-series classifier to effectively learn the difference between safe and dangerous conditions. The inclusion of fatigue-related features and proper class balancing further enhances the classifier’s ability to accurately predict injury-prone moments, which is crucial for a reliable injury warning system.

## Conclusion  
In summary, the analysis identified specific outcome measures for both muscle fatigue and injury risk that can serve as targets for predictive modeling. For fatigue forecasting, muscle-level spectral and amplitude indices (like median frequency and RMS EMG) and joint performance measures provide a quantitative gauge of exhaustion. For injury classification, biomechanical red flags such as knee valgus loading, high impact forces, and coordination breakdown mark the moments of elevated risk. We engineered a comprehensive set of features from the EMG and kinematic data – spanning time, frequency, and time-frequency domains – to feed into models. An LSTM-based regression approach was recommended for modeling fatigue progression over time, taking advantage of temporal patterns in the data, while a time-series classification framework (with careful windowing and balance) was outlined for predicting injury events. These methods, supported by domain knowledge and literature, offer a structured way to use the given dataset to anticipate fatigue and potential injuries, ultimately aiding in proactive interventions.

